## Imports 

In [1]:
import numpy as np
import cvxpy as cp
from scipy.optimize import minimize

import random

### Solar Value Generation

In [2]:
import numpy as np

def generate_daily_solar_profile(sunrise=6, sunset=20, mean_pmax=15, std_pmax=1, seed=None):
    """
    Generate a 24-hour solar irradiance profile using a cosine model for daylight hours.
    
    Parameters:
    - sunrise: Hour of sunrise (e.g., 6)
    - sunset: Hour of sunset (e.g., 18)
    - mean_pmax: Mean of P_max (e.g., 1000 W/m²)
    - std_pmax: Standard deviation of P_max (e.g., 100 W/m²)
    - seed: Random seed for reproducibility
    
    Returns:
    - irradiance: numpy array of length 24, representing hourly solar irradiance
    - p_max: the actual peak irradiance drawn for the day
    """
    if seed is not None:
        np.random.seed(seed)
    
    hours = np.arange(24 + 1)  # 0 to 25 
    irradiance = np.zeros_like(hours, dtype=float)
    
    t_mid = (sunrise + sunset) / 2
    daylight_duration = sunset - sunrise

    # Draw P_max from normal distribution
    p_max = np.random.normal(loc=mean_pmax, scale=std_pmax)

    # Apply cosine model during daylight hours
    for h in range(24 + 1):
        
        if sunrise <= h <= sunset:
            val = p_max * np.cos(np.pi * (h - t_mid) / daylight_duration)
            irradiance[h] = max(0, val)  # clip negative values to zero

        #ensuring that the 25th hour is the same as the 0th hour
        if (h == 25): val[h] == val[0] 

    return irradiance, p_max

# Example usage
# irradiance_profile, p_max = generate_daily_solar_profile(seed=0)
# print(f"P_max for the day: {p_max:.2f}")
# print("Hourly irradiance values:")
# print(irradiance_profile)

def generate_nday_solar(sunrise=6, sunset=20, mean_pmax=14, std_pmax=1, days= 1):
    """Generate nday solar production"""

    irradiance_profile = []
    p_max_list = []
    for day in range(days): 
        #print(f"Day = {day}")
        power_gen_day, p_max = generate_daily_solar_profile(sunrise, sunset, mean_pmax, std_pmax, day)
        #print(f"P_max for the day: {p_max:.2f}")
        #print("Hourly irradiance values:")

        if day == 0: #we want 25 values for day 1 
            irradiance_profile.extend(power_gen_day)
        else: #we want less than 25 values for day 2 
            irradiance_profile.extend(power_gen_day[1:])

        p_max_list.append(p_max)
    
    return irradiance_profile, p_max

irradiance_profile, p_max = generate_nday_solar(sunrise=6, sunset=20, mean_pmax=14, std_pmax = 1, days= 7)

print(f"irradiance_profile = {len(irradiance_profile)}")
        

irradiance_profile = 169


### Perfectly Elastic Demand Model

In [37]:
T = 24*2 + 1
v_d = 10
c_u = 5
q_d = np.ones(T)*10
s_t = np.array(irradiance_profile)  
C_init = 2
C_final = 2
C_max = 10

#np.random.seed(0)
# === Variable unpacking ===
def unpack(x):
    p = x[0:T]
    q_s = x[T:2*T]
    q_b = x[2*T:3*T]
    q_u = x[3*T:4*T]
    C = x[4*T:5*T+1]
    return p, q_s, q_b, q_u, C

# === Objective ===
def objective(x):
    p, q_s, q_b, q_u, _ = unpack(x)
    term1 = (v_d - p) * q_d       # consumer surplus
    term2 = p * (q_s + q_b)       # payments to solar and battery
    term3 = (p - c_u) * q_u       # utility surplus
    #return -np.sum(term1 + term2 + term3)
    return -np.sum(term1 + term2 + term3)

# === Constraints ===
def constraint_eq(x):
    p, q_s, q_b, q_u, C = unpack(x)
    cons = []

    # Initial battery
    cons.append(C[0] - C_init)
    cons.append(C[T] - C_final)        

    # Final hour values match initial hour values (cyclic constraint)
    cons.append(q_s[0] - q_s[-1])
    cons.append(q_b[0] - q_b[-1])
    cons.append(q_u[0] - q_u[-1])
    cons.append(p[0] - p[-1])

    for t in range(T):
        # Market clearing
        cons.append(q_s[t] + q_b[t] + q_u[t] - q_d[t])
        
        # Battery state update
        cons.append(C[t+1] - (C[t] - q_b[t]))

    return np.array(cons)


def constraint_ineq(x):
    p, q_s, q_b, q_u, _ = unpack(x)
    ineqs = []

    # Per-period IR constraints for consumer, utility, solar
    for t in range(T):
        ineqs.append(s_t[t] - q_s[t])        # solar capacity
        ineqs.append((v_d - p[t]) * q_d[t])     # Consumer
        ineqs.append((p[t] - c_u) * q_u[t])     # Utility
        ineqs.append(p[t] * q_s[t])             # Solar

    # Aggregate IR constraint for battery
    IR_b = np.sum(p * q_b)
    ineqs.append(IR_b)

    return np.array(ineqs)


# === Bounds ===
bounds = (
    [(0, None)] * T +       # Prices ≥ 0
    [(0, None)] * T +       # Solar quantities ≥ 0
    [(-C_max, C_max)] * T + # Battery can charge (-) or discharge (+)
    [(0, None)] * T +       # Utility quantities ≥ 0
    [(0, C_max)] * (T + 1)  # Battery state of charge
)


# === Initial Guess ===
x0 = np.zeros(5*T + 1)*c_u
x0[4*T] = C_init  # battery start
x0[5*T] = C_final
# === Solve ===
res = minimize(
    fun=objective,
    x0=x0,
    method='SLSQP',
    bounds=bounds,
    constraints=[
        {'type': 'eq', 'fun': constraint_eq},
        {'type': 'ineq', 'fun': constraint_ineq}
    ],
    options={'disp': True, 'maxiter': 1000}
)

if res.success:
    p, q_s, q_b, q_u, C = unpack(res.x)
    print("Optimal Welfare:", -res.fun)
else:
    print("Optimization failed:", res.message)


Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: -4900.0
            Iterations: 5
            Function evaluations: 247
            Gradient evaluations: 1
Optimization failed: Positive directional derivative for linesearch


In [70]:
import plotly.graph_objs as go
import plotly.offline as pyo

fig = go.Figure()
fig.add_trace(go.Scatter(y=p, mode='lines', name='Price p'))
fig.update_layout(
    title='Price (p) over Time Steps',
    xaxis_title='Time Step',
    yaxis_title='Price (p)'
)
fig.show()


In [39]:
import plotly.graph_objs as go

fig = go.Figure()
fig.add_trace(go.Scatter(y=q_d, mode='lines', name='q_d'))
fig.add_trace(go.Scatter(y=q_b, mode='lines', name='q_b'))
fig.add_trace(go.Scatter(y=q_s, mode='lines', name='q_s'))
fig.add_trace(go.Scatter(y=q_u, mode='lines', name='q_u'))
fig.add_trace(go.Scatter(y=C, mode='lines', name='C'))
fig.add_trace(go.Scatter(y=s_t, mode='lines', name='s_t'))
fig.update_layout(
    title='Quantity of Energy Dispatched over Time Steps',
    xaxis_title='Time Step',
    yaxis_title='Q (kwh)'
)
fig.show()


ValueError: 
    Invalid value of type 'numpy.float64' received for the 'y' property of scatter
        Received value: 1.0

    The 'y' property is an array that may be specified as a tuple,
    list, numpy array, or pandas Series

### Introducing Demand Elasticity 

We introduce the model some demand elasticity, the issue so far is that when there is the possibility of 0 prices 
there are no 0 prices because v_d > 0 for all quantities purchased, instead, we are going to treat it as 
aggregate demand so I we can get a downward slope. 

$v(q) = v_{max}*(1 - \frac{q}{q_{max}})$

and the total surplus given a price $p$ is equa

### Plotting Consumer surplus under DA 

In [1]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact, FloatSlider

def demand_curve(q, v_max, q_max):
    return v_max * (1 - q / q_max)

def consumer_surplus(p, v_max, q_max):
    if p >= v_max:
        return 0, 0
    q_star = q_max * (1 - p / v_max)
    area = v_max * q_star - (v_max / (2 * q_max)) * q_star**2 - p * q_star
    return area, q_star

def plot_demand_curve_plotly(v_max=0.4, q_max=10.0, price=0.2):
    q_vals = np.linspace(0, q_max, 300)
    v_vals = demand_curve(q_vals, v_max, q_max)
    area, q_star = consumer_surplus(price, v_max, q_max)

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=q_vals, y=v_vals, mode='lines', name='Inverse Demand Curve'))
    fig.add_trace(go.Scatter(x=[0, q_max], y=[price, price], mode='lines',
                             name='Market Price', line=dict(dash='dash', color='red')))

    q_fill = q_vals[q_vals <= q_star]
    v_fill = demand_curve(q_fill, v_max, q_max)
    fig.add_trace(go.Scatter(
        x=np.concatenate([q_fill, q_fill[::-1]]),
        y=np.concatenate([v_fill, [price]*len(q_fill)]),
        fill='toself',
        fillcolor='rgba(255,165,0,0.5)',
        line=dict(color='orange'),
        name='Consumer Surplus'
    ))

    fig.update_layout(
        title=f'Consumer Surplus = {area:.4f}, Equilibrium Quantity = {q_star:.2f}',
        xaxis_title='Quantity (q)',
        yaxis_title='Willingness to Pay (v(q))',
        xaxis=dict(range=[0, q_max*2]),
        yaxis=dict(range=[0, 15]),
        legend=dict(x=0.01, y=0.99)
    )

    fig.show()

interact(plot_demand_curve_plotly,
         v_max=FloatSlider(value=0.4, min=0.1, max=10, step=0.01, description='v_max'),
         q_max=FloatSlider(value=10, min=1, max=20, step=0.5, description='q_max'),
         price=FloatSlider(value=0.2, min=0.0, max=10, step=0.01, description='Price'));


interactive(children=(FloatSlider(value=0.4, description='v_max', max=10.0, min=0.1, step=0.01), FloatSlider(v…

### Plotting Consumer Surplus under AMM Marginal Cost

In [ ]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact, FloatSlider

def demand_curve(q, v_max, q_max):
    """
    Inverse demand (marginal value) function v(q).
    """
    return v_max * (1 - q / q_max)


def amm_supply_curve(q, E, M):
    """
    AMM marginal cost supply curve: price to purchase an incremental unit of E.
    Derived as dM/de = K/(E - e)^2, where K = E*M.
    """
    K = E * M
    return K / (E - q)**2


def plot_demand_and_amm_supply(v_max=0.4, q_max=10.0, price=0.2, E=10.0, M=10.0):
    # Quantity grid up to q_max
    q_vals = np.linspace(0, q_max, 300)

    # Demand curve (marginal value)
    v_vals = demand_curve(q_vals, v_max, q_max)

    # AMM supply (marginal cost)
    supply_vals = amm_supply_curve(q_vals, E, M)

    # Compute consumer surplus at given price
    area, q_star = consumer_surplus(price, v_max, q_max)

    fig = go.Figure()

    # Plot demand curve
    fig.add_trace(go.Scatter(
        x=q_vals, y=v_vals,
        mode='lines', name='Inverse Demand Curve'
    ))

    # Plot AMM supply curve
    fig.add_trace(go.Scatter(
        x=q_vals, y=supply_vals,
        mode='lines', name='AMM Supply Curve',
        line=dict(dash='dot', color='green')
    ))

    # Plot market price line
    fig.add_trace(go.Scatter(
        x=[0, q_max], y=[price, price],
        mode='lines', name='Market Price',
        line=dict(dash='dash', color='red')
    ))

    # Shade consumer surplus
    q_fill = q_vals[q_vals <= q_star]
    v_fill = demand_curve(q_fill, v_max, q_max)
    fig.add_trace(go.Scatter(
        x=np.concatenate([q_fill, q_fill[::-1]]),
        y=np.concatenate([v_fill, [price]*len(q_fill)]),
        fill='toself', fillcolor='rgba(255,165,0,0.5)',
        line=dict(color='orange'), name='Consumer Surplus'
    ))

    fig.update_layout(
        title=f'Consumer Surplus = {area:.4f}, Equilibrium Quantity = {q_star:.2f}',
        xaxis_title='Quantity (q)',
        yaxis_title='Price / Marginal Value',
        xaxis=dict(range=[0, q_max*1.1]),
        yaxis=dict(range=[0, max(v_max, max(supply_vals))*1.1]),
        legend=dict(x=0.01, y=0.99)
    )

    fig.show()

# Interactive sliders to adjust parameters
interact(
    plot_demand_and_amm_supply,
    v_max=FloatSlider(value=0.4, min=0.1, max=100, step=0.01, description='v_max'),
    q_max=FloatSlider(value=10, min=1, max=20, step=0.5, description='q_max'),
    price=FloatSlider(value=0.2, min=0.0, max=10, step=0.01, description='Price'),
    E=FloatSlider(value=10, min=1, max=100, step=1, description='E reserve'),
    M=FloatSlider(value=10, min=1, max=100, step=1, description='M reserve')
)


interactive(children=(FloatSlider(value=0.4, description='v_max', min=0.1, step=0.01), FloatSlider(value=10.0,…

<function __main__.plot_demand_and_amm_supply(v_max=0.4, q_max=10.0, price=0.2, E=10.0, M=10.0)>

In [ ]:
T = 48
v_d = 0.2
c_u = 0.1
q_d = np.ones(T)*5 
s_t = np.array(irradiance_profile)  
C_init = 2
C_final = 2
C_max = 10
v_max = 0.4
q_max = 10



# === Variable unpacking ===
def unpack(x):
    p = x[0:T]
    q_s = x[T:2*T]
    q_b = x[2*T:3*T]
    q_u = x[3*T:4*T]
    C = x[4*T:5*T+1]
    return p, q_s, q_b, q_u, C


# === Objective ===
def objective(x, v_max=0.4, q_max=10.0, c_u=0.1):
    p, q_s, q_b, q_u, q_d, _ = unpack(x)
    
    # Consumer total utility (area under demand curve)
    consumer_utility = v_max * q_d - (v_max / (2 * q_max)) * q_d**2 - p*q_d

    # Payments to solar and battery
    producer_revenue = p * (q_s + q_b)

    # Utility surplus
    utility_profit = (p - c_u) * q_u

    return -np.sum(consumer_utility + producer_revenue + utility_profit)


# === Constraints ===
def constraint_eq(x):
    p, q_s, q_b, q_u, C = unpack(x)
    cons = []

    # Initial battery
    cons.append(C[0] - C_init)
    cons.append(C[T] - C_final)        # Ending charge
    for t in range(T):
        # Market clearing
        q_d_t = q_max * (1 - p[t] / v_max)
        cons.append(q_s[t] + q_b[t] + q_u[t] - q_d_t)

        #cons.append(q_s[t] + q_b[t] + q_u[t] - q_d[t])
        
        # Battery state update
        cons.append(C[t+1] - (C[t] - q_b[t]))


    return np.array(cons)

# def constraint_ineq(x):
#     p, q_s, q_b, q_u, _ = unpack(x)
#     ineqs = []

#     # Solar + battery limits
#     ineqs += list(s_t - q_s)      # q_s <= s_t
#     # ineqs += list(q_s)            # q_s >= 0
#     # ineqs += list(q_u)            # q_u >= 0
#     # ineqs += list(p)              # p >= 0

#     # IR constraints:
#     IR_d = np.sum((v_d - p) * q_d)
#     IR_u = np.sum((p - c_u) * q_u)
#     IR_s = np.sum(p * q_s)
#     IR_b = np.sum(p * q_b)

#     ineqs += [IR_d, IR_u, IR_s, IR_b]

#     return np.array(ineqs)

def constraint_ineq(x):
    p, q_s, q_b, q_u, _ = unpack(x)
    ineqs = []

    # Physical feasibility
    ineqs += list(s_t - q_s)  # solar limit: q_s ≤ s_t
    # ineqs += list(q_s)        # q_s ≥ 0
    # ineqs += list(q_u)        # q_u ≥ 0
    # ineqs += list(p)          # p ≥ 0

    # Per-period IR constraints for consumer, utility, solar
    for t in range(T):

        q_d_t = q_max * (1 - p[t] / v_max)
        IR_d_t = v_max * q_d_t - (v_max / (2 * q_max)) * q_d_t**2 - p[t] * q_d_t
        ineqs.append(IR_d_t)

        ineqs.append((v_d - p[t]) * q_d[t])     # Consumer
        ineqs.append((p[t] - c_u) * q_u[t])     # Utility
        ineqs.append(p[t] * q_s[t])             # Solar

    # Aggregate IR constraint for battery
    IR_b = np.sum(p * q_b)
    ineqs.append(IR_b)

    return np.array(ineqs)


# === Bounds ===
bounds = [(0, None)] * (4*T) + [(0, C_max)] * (T+1)

# === Initial Guess ===
x0 = np.ones(5*T + 1)*c_u
x0[4*T] = C_init  # battery start
x0[5*T] = C_final
# === Solve ===
res = minimize(
    fun=objective,
    x0=x0,
    method='SLSQP',
    bounds=bounds,
    constraints=[
        {'type': 'eq', 'fun': constraint_eq},
        {'type': 'ineq', 'fun': constraint_ineq}
    ],
    options={'disp': True, 'maxiter': 1000}
)

if res.success:
    p, q_s, q_b, q_u, C = unpack(res.x)
    print("Optimal Welfare:", -res.fun)
else:
    print("Optimization failed:", res.message)


## Elastic Demand Optimization

In [78]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd


# === Parameters ===
days = 7
T = 24*days + 1
v_max = 10
q_max = 10
c_u = 5
C_init = 2
C_final = 2
C_max = 10

# === Simulated solar irradiance ===
sunrise = 6 
sunset = 20
mean_pmax = 10
std_pmax = 0

solar_gen_profile, p_max_list = generate_nday_solar(sunrise, sunset, mean_pmax, std_pmax, days)

#np.random.seed(0)
s_t = np.array(solar_gen_profile)  
#s_t = np.zeros(T)

# === Unpacking function ===
def unpack(x):
    p = x[0:T]
    q_s = x[T:2*T]
    q_b = x[2*T:3*T]
    q_u = x[3*T:4*T]
    C = x[4*T:5*T+1]
    return p, q_s, q_b, q_u, C

# === Objective function ===
def objective(x):
    p, q_s, q_b, q_u, _ = unpack(x)
    q_d = q_max * (1 - p / v_max)

    # Consumer surplus (area under demand curve minus expenditure)
    cs = v_max * q_d - (v_max / (2 * q_max)) * q_d**2 - p * q_d

    # Payments to solar + battery
    producer_rev = p * q_s
    battery_rev = p*q_b

    # Utility company profit
    utility_profit = (p - c_u) * q_u

    return -np.sum(cs + producer_rev + utility_profit + battery_rev)
    
   

# === Equality Constraints ===
def constraint_eq(x):
    p, q_s, q_b, q_u, C = unpack(x)
    cons = []

    cons.append(C[0] - C_init)
    cons.append(C[T] - C_final) 

    # Final hour values match initial hour values (cyclic constraint)
    cons.append(q_s[0] - q_s[-1])
    cons.append(q_b[0] - q_b[-1])
    cons.append(q_u[0] - q_u[-1])


    for t in range(T):
        q_d_t = q_max * (1 - p[t] / v_max)
        cons.append(q_s[t] + q_b[t] + q_u[t] - q_d_t)
        cons.append(C[t+1] - (C[t] - q_b[t]))
    
    return np.array(cons)

# === Inequality Constraints ===
def constraint_ineq(x):
    p, q_s, q_b, q_u, _ = unpack(x)
    ineqs = []

    for t in range(T):
        q_d_t = q_max * (1 - p[t] / v_max)

        # Physical limits
        ineqs.append(s_t[t] - q_s[t])        # solar capacity
        # ineqs.append(q_s[t])                 # solar ≥ 0
        # ineqs.append(q_u[t])                 # utility ≥ 0
        # ineqs.append(p[t])                   # price ≥ 0

        # IR constraints per period
        IR_d_t = v_max * q_d_t - (v_max / (2 * q_max)) * q_d_t**2 - p[t] * q_d_t
        ineqs.append(IR_d_t)
        ineqs.append((p[t] - c_u) * q_u[t])  # utility IR
        ineqs.append(p[t] * q_s[t])          # solar IR

    # Battery IR across time
    IR_b = np.sum(p * q_b)
    ineqs.append(IR_b)

    return np.array(ineqs)

# === Bounds ===
bounds = (
    [(0, None)] * T +       # Prices ≥ 0
    [(0, None)] * T +       # Solar quantities ≥ 0
    [(-C_max, C_max)] * T + # Battery can charge (-) or discharge (+)
    [(0, None)] * T +       # Utility quantities ≥ 0
    [(0, C_max)] * (T + 1)  # Battery state of charge
)


# === Initial Guess ===
x0 = np.ones(5*T + 1) * c_u
# x0[:T] *= c_u  # prices initially set to marginal cost
# x0[T:2*T] = s_t   # solar dispatch initially matches solar availability
x0[4*T] = C_init  # Battery initial SoC
x0[2*T:3*T] = 0   # Start neutral, let optimizer decide charge/discharge
x0[5*T] = C_final

#===Educated Guess====
x0 = np.ones(5*T + 1)
x0[:T] *= c_u # prices initially set to marginal cost
x0[T:2*T] = s_t   # solar dispatch initially matches solar availability
x0[2*T:3*T] = 0    # battery initially neutral (no dispatch)
x0[3*T:4*T] = np.min(abs(5 - s_t), 0) # utility initially covers residual
x0[4*T:5*T+1] = C_init  # Battery SOC initially constant


# === Solve ===
res = minimize(
    fun=objective,
    x0=x0,
    method='SLSQP',
    bounds=bounds,
    constraints=[
        {'type': 'eq', 'fun': constraint_eq},
        {'type': 'ineq', 'fun': constraint_ineq}
    ],
    options={'disp': True, 'maxiter': 8000}
)

# === Output ===
if res.success:
    p, q_s, q_b, q_u, C = unpack(res.x)
    q_d = q_max * (1 - p / v_max)
    df_opt = pd.DataFrame({
        'Hour': np.arange(T),
        'Price': p,
        'q_d': q_d,
        'q_s': q_s,
        'q_b': q_b,
        'q_u': q_u,
        'Battery': C[:-1]
    })
    print(df_opt.head())
    print(f"\nOptimal Welfare: {-res.fun:.3f}")

    # Check which constraints are binding
    eq_vals = constraint_eq(res.x)
    ineq_vals = constraint_ineq(res.x)
    tol = 1e-6  # tolerance for binding

    binding_eq = np.where(np.abs(eq_vals) > tol)[0]
    binding_ineq = np.where(np.abs(ineq_vals) < tol)[0]

    print(f"\nBinding equality constraints indices: {binding_eq}")
    print(f"Binding inequality constraints indices: {binding_ineq}")
else:
    print("Optimization failed:", res.message)


Optimization terminated successfully    (Exit mode 0)
            Current function value: -4530.861453822827
            Iterations: 3
            Function evaluations: 2552
            Gradient evaluations: 3
   Hour  Price  q_d  q_s       q_b       q_u   Battery
0     0    5.0  5.0  0.0  0.057015  4.942985  2.000000
1     1    5.0  5.0  0.0  0.005764  4.994236  1.942985
2     2    5.0  5.0  0.0  0.037153  4.962847  1.937221
3     3    5.0  5.0  0.0  0.087119  4.912881  1.900068
4     4    5.0  5.0  0.0  0.180645  4.819355  1.812949

Optimal Welfare: 4530.861

Binding equality constraints indices: []
Binding inequality constraints indices: [  0   2   3   4   6   7   8  10  11  12  14  15  16  18  19  20  22  23
  24  26  27  28  30  32  36  38  42  46  50  54  58  62  66  68  70  72
  74  76  78  80  82  83  84  86  87  88  90  91  92  94  95  96  98  99
 100 102 103 104 106 107 108 110 111 112 114 115 116 118 119 120 122 123
 124 126 128 132 134 138 142 146 150 154 158 162 164 166 16

In [79]:
df_opt_rounded = df_opt.round(4)
df_opt_rounded

,Hour,Price,q_d,q_s,q_b,q_u,Battery
0,0,5.0,5.0,0.0,0.0570,4.9430,2.0000
1,1,5.0,5.0,0.0,0.0058,4.9942,1.9430
2,2,5.0,5.0,0.0,0.0372,4.9628,1.9372
3,3,5.0,5.0,0.0,0.0871,4.9129,1.9001
4,4,5.0,5.0,0.0,0.1806,4.8194,1.8129
...,...,...,...,...,...,...,...
164,164,5.0,5.0,0.0,1.0381,3.9619,4.0626
165,165,5.0,5.0,0.0,0.5258,4.4742,3.0245
166,166,5.0,5.0,0.0,0.2764,4.7236,2.4987
167,167,5.0,5.0,0.0,0.1653,4.8347,2.2223


In [80]:
import plotly.graph_objs as go
import plotly.offline as pyo

fig = go.Figure()
fig.add_trace(go.Scatter(y=p, mode='lines', name='Price p'))
fig.update_layout(
    title='Price (p) over Time Steps',
    xaxis_title='Time Step',
    yaxis_title='Price (p)'
)
fig.show()


In [45]:
max(s_t)

2.0

In [81]:
import plotly.graph_objs as go

fig = go.Figure()
fig.add_trace(go.Scatter(y=q_d, mode='lines', name='q_d'))
fig.add_trace(go.Scatter(y=q_b, mode='lines', name='q_b'))
fig.add_trace(go.Scatter(y=q_s, mode='lines', name='q_s'))
fig.add_trace(go.Scatter(y=q_u, mode='lines', name='q_u'))
fig.add_trace(go.Scatter(y=C, mode='lines', name='C'))
fig.add_trace(go.Scatter(y=s_t, mode='lines', name='s_t'))
fig.update_layout(
    title='Quantity of Energy Dispatched over Time Steps',
    xaxis_title='Time Step',
    yaxis_title='Q (kwh)'
)
fig.show()


In [47]:
import numpy as np

T = 24
beta = 1.0  # no discounting, or use beta < 1
C_grid = np.linspace(0, 5, 50)
s_grid = np.linspace(0.2, 0.8, 5)  # or treat as exogenous

V = np.zeros((len(C_grid), T+1))  # value function V(C, t)

# Backward iteration
for t in reversed(range(T)):
    for i, C in enumerate(C_grid):
        for s in s_grid:
            best_value = -np.inf
            for q_b in np.linspace(-1, 1, 20):  # battery actions
                if 0 <= C - q_b <= C_max:
                    q_s = min(s, 1.0)
                    q_u = max(0, 1.0 - q_s - q_b)
                    p = compute_price(q_s, q_b, q_u)  # based on equilibrium, or trial value
                    inst_welfare = (v_d - p)*1.0 + p*q_s + p*q_b + (p - c_u)*q_u
                    C_next = C - q_b
                    i_next = np.argmin(np.abs(C_grid - C_next))
                    value = inst_welfare + beta * V[i_next, t+1]
                    best_value = max(best_value, value)
            V[i, t] = best_value



NameError: name 'compute_price' is not defined

## 24-Period AMM Model (Daily)

The goal here is to figure out, or get a sense of where should the liquidity provider   
add liquidity so that they don't lose money. Once that can be figured out, the problem might be more straightforward   
  
We are assuming a simple timestep of 1 hour for now (remember this is arbitrary, everything can scaled down to 5 mins be dividing by 12)

In [ ]:
class Demand: 
    def __init__(self, v_d, q_d, AMM=None, verbose=False): 
        # state variables 
        self.v_d = v_d # per kwh reservation price 
        self.d = q_d # kwh period demand in the current period
        self.remaining_demand = q_d # tracks how much demand has been bought and sold

        # welfare 
        self.profit = 0 
        self.sw_arr = []

        # AMM Instance 
        self.AMM = AMM # NOTE: return here

        # txs request 
        self.txs_request = None 

        # debugging
        self.verbose = verbose

        print(f"[{self.agent_type}][INIT] v_d={v_d}, q_d={q_d}, AMM={AMM}, verbose={verbose}")

    def update_profit(self, p, q): 
        utils = self.util_func(p, q)
        self.profit += utils
        self.sw_arr.append(utils)
        print(f"[{self.agent_type}][update_profit] Updated profit by {utils}, total profit now {self.profit}")
        if self.verbose:
            print(f"[{self.agent_type}][update_profit][VERBOSE] p={p}, q={q}, utils={utils}, sw_arr={self.sw_arr}")

    def util_func(self, p, q):
        result = (self.v_d - p) * q
        if self.verbose:
            print(f"[{self.agent_type}][util_func][VERBOSE] v_d={self.v_d}, p={p}, q={q}, result={result}")
        return result
    
    def make_decision(self, E, M):
        print(f"[{self.agent_type}][make_decision] Called with E={E}, M={M}")
        q_d = self.remaining_demand # tracks the amount of demand not purchased
        print(f"[{self.agent_type}][make_decision] Remaining demand: {q_d}")

        if round(q_d, 12) > 0: # if there is positive demand remaining
            exp_total_amm_price = self.calc_price(E, M, q_d) 
            print(f"[{self.agent_type}][make_decision] Expected AMM price for {q_d}: {exp_total_amm_price}")

            if exp_total_amm_price > (self.v_d * q_d): 
                e_req = E - (M / self.v_d) # NOTE: could be wrong and needs to be tested 
                print(f"[{self.agent_type}][make_decision] AMM price too high, calculating e_req: {e_req}")

                if e_req <= 0:
                    print("[{self.agent_type}][make_decision] Not enough liquidity to buy. Skipping trade.")
                    return None

                exp_total_amm_price = self.calc_price(E, M, e_req)
                print(f"[{self.agent_type}][make_decision] New expected AMM price for e_req={e_req}: {exp_total_amm_price}")

                txs_request = {
                    "decision": "buy_tokens_max_price",
                    "token": "x",
                    "quantity": e_req,
                    "max_price": self.v_d * e_req
                }
                if self.verbose:
                    print(f"[{self.agent_type}][make_decision][VERBOSE] txs_request={txs_request}")
            else: 
                e_req = q_d
                txs_request = {
                    "decision": "buy_tokens_max_price",
                    "token": "x",
                    "quantity": q_d,
                    "max_price": self.v_d * e_req
                }
                if self.verbose:
                    print(f"[{self.agent_type}][make_decision][VERBOSE] txs_request={txs_request}")

            print(f"[{self.agent_type}][make_decision] Final txs_request: {txs_request}")
            self.txs_request = txs_request
            return txs_request

        else: 
            print("[{self.agent_type}][make_decision] No remaining demand.")
            return 
              

    def calc_price(self, E, M, e): 
        if self.verbose:
            print(f"[{self.agent_type}][calc_price][VERBOSE] E={E}, M={M}, e={e}")
        if abs(E - e) < 1e-12:
            print("[{self.agent_type}][calc_price] Requesting entire pool, returning inf cost.")
            return float("inf")  # requesting entire pool => infinite cost
        k = E * M 
        m = k / (E - e) - M
        if self.verbose:
            print(f"[{self.agent_type}][calc_price][VERBOSE] k={k}, m={m}")
        return m

    def transaction_result(self, txs_result): 
        print(f"[{self.agent_type}][transaction_result] txs_result: {txs_result}")
        e_requested = self.txs_request["quantity"]
        m_required = txs_result["quantity_needed"]
        avg_price = m_required / e_requested

        print(f"[{self.agent_type}][transaction_result] avg_price = {avg_price}")

        self.update_profit(avg_price, e_requested)
        self.remaining_demand -= e_requested
        print(f"[{self.agent_type}][transaction_result] Updated remaining_demand: {self.remaining_demand}")



class Supply: 
    def __init__(self, c, q_sell, AMM, verbose=False): 
        # state variables 
        self.c = c # per kwh reservation price 
        self.q_sell = q_sell # kwh period demand in the current period
        self.remaining_supply = q_sell # tracks how much demand has been bought and sold

        # welfare 
        self.profit = 0 
        self.sw_arr = []

        # AMM Instance 
        self.AMM = AMM # NOTE: return here

        # txs request 
        self.txs_request = None 

        # debugging
        self.verbose = verbose

        print(f"[{self.agent_type}][INIT] c={c}, q_sell={q_sell}, AMM={AMM}, verbose={verbose}")

    def update_profit(self, p, q): 
        utils = self.util_func(p, q)
        self.profit += utils
        self.sw_arr.append(utils)
        print(f"[{self.agent_type}][update_profit] Updated profit by {utils}, total profit now {self.profit}")
        if self.verbose:
            print(f"[{self.agent_type}][update_profit][VERBOSE] p={p}, q={q}, utils={utils}, sw_arr={self.sw_arr}")

    def make_decision(self, E, M):
        print(f"[{self.agent_type}][make_decision] Called with E={E}, M={M}")
        q_s = self.remaining_supply # tracks the amount of supply not purchased
        print(f"[{self.agent_type}][make_decision] Remaining supply: {q_s}")

        if round(q_s, 12) > 0: # if there is positive supply remaining
            exp_total_amm_price = self.calc_price(E, M, q_s)
            print(f"[{self.agent_type}][make_decision] Expected AMM price for {q_s}: {exp_total_amm_price}")

            if exp_total_amm_price < (self.c * q_s): 
                e_sell = (M / self.c) - E # NOTE: could be wrong and needs to be tested 
                print(f"[{self.agent_type}][make_decision] AMM price too low, calculating e_sell: {e_sell}")
                exp_total_amm_price = self.calc_price(E, M, e_sell)

                if e_sell < 0:
                    print("[{self.agent_type}][make_decision] No profitable trades remaining.")
                    return # no profitable trades remaining

                txs_request = {
                    "decision": "sell_tokens_min_price",
                    "token": "x",
                    "quantity": e_sell,
                    "min_price": self.c * e_sell
                }
                if self.verbose:
                    print(f"[{self.agent_type}][make_decision][VERBOSE] txs_request={txs_request}")
            else: 
                e_sell = q_s
                txs_request = {
                    "decision": "sell_tokens_min_price",
                    "token": "x",
                    "quantity": e_sell,
                    "min_price": self.c * e_sell
                }
                if self.verbose:
                    print(f"[{self.agent_type}][make_decision][VERBOSE] txs_request={txs_request}")

            print(f"[{self.agent_type}][make_decision] Final txs_request: {txs_request}")
            self.txs_request = txs_request
            return txs_request

        else: 
            print("[{self.agent_type}][make_decision] No remaining supply.")
            return 
        
    def transaction_result(self, txs_result): 
        print(f"[{self.agent_type}][transaction_result] txs_result: {txs_result}")
        e_sell = self.txs_request["quantity"]
        m_returned = txs_result["quantity_returned"]
        avg_price = m_returned / e_sell

        print(f"[{self.agent_type}][transaction_result] avg_price = {avg_price}, self.c = {self.c}")

        self.update_profit(avg_price, e_sell)
        self.remaining_supply -= e_sell
        print(f"[{self.agent_type}][transaction_result] Updated remaining_supply: {self.remaining_supply}")

    def util_func(self, p, q):
        result = (p - self.c) * q
        if self.verbose:
            print(f"[{self.agent_type}][util_func][VERBOSE] p={p}, c={self.c}, q={q}, result={result}")
        return result
    
    def calc_price(self, E, M, e): 
        if self.verbose:
            print(f"[{self.agent_type}][calc_price][VERBOSE] E={E}, M={M}, e={e}")
        k = E * M 
        m = M - k / (E + e)
        if self.verbose:
            print(f"[{self.agent_type}][calc_price][VERBOSE] k={k}, m={m}")
        return m


### Uniswap Class 

In [ ]:
import numpy as np 

class AMM(object):
    """
    In the simple model below, 
    x tokens represent Energy tokens or E_tokens
    y tokens represent Money tokens or M_tokens 
    """
    def __init__(self,transaction_fee = 0, ratio_error_tol = 1e-7, debug=False):
        
        self.reserve_x = 0
        self.reserve_y = 0
        
        self.constant_product = 0
        self.governance_tokens = 0
        self.lp_tokens = 0 
        
        self.transaction_fee = transaction_fee
        
        self.debug = debug
        self.ratio_error_tol = ratio_error_tol
        
    def setup_pool(self, quantity_x=0, quantity_y=0):
        """
        Establishes a liquidity pool with constant_product = quantity_x * quantity_y.
        Returns liqudity tokens to the agent who sets it up. 
        See 1.A "Establish Liquidity" above for explanation of math. 
        """
        self.reserve_x  = quantity_x
        self.reserve_y = quantity_y
        
        self.constant_product = self.reserve_x*self.reserve_y
        
        lp_minted  = np.sqrt(self.constant_product)
        self.lp_tokens = lp_minted
        
        return lp_minted 
        
    def request_info(self):
        """
        Simple method that informs an agent of the current reserve amounts of 
        X and Y in the liquidity pool. 
        See 1.B "Get Current Amount of Reserves" above for explanation of math. 
        """
        info_dict ={"reserve_x": self.reserve_x, "reserve_y": self.reserve_y,
                       "transaction_fee": self.transaction_fee}
        return info_dict
    
    def provide_liquidity(self,quantity_x, quantity_y): 
        """
        Allows agents to add tokens X and Y to the liquidity pool. 
        Agents must submit X and Y in the exact ratio of the current reserves. 
        Otherwise, method returns an error. 
        If successful, agents receive liquidity tokens in response. 
        See 1.C "Add Liquidity" for an explanation of math. 
        """
        ratio_submitted = quantity_x/quantity_y 
        current_reserve_ratio = self.reserve_x/self.reserve_y
        
        
        if self.debug:
            print(f"-"*10, f"PROVIDE LIQUIDITY ",f"-"*10)
            print(f"-"*10, f"PRIOR LIQUIDITY ",f"-"*10)
            print(f"Reserve X:{self.reserve_x}\n"+
                 f"Reserve Y:{self.reserve_y}\n" +
                 f"Total pool tokens :{self.lp_tokens}")
        
        
        if abs(ratio_submitted - current_reserve_ratio) < self.ratio_error_tol:
            
            lp_minted = (quantity_x/self.reserve_x)*self.lp_tokens
            
            self.lp_tokens = self.lp_tokens + lp_minted 
            self.reserve_x = self.reserve_x + quantity_x 
            self.reserve_y = self.reserve_y + quantity_y 
            
            if self.debug:
                print(f"-"*10, f"AFTER LIQUIDITY INSERTED",f"-"*10)
                print(f"Reserve X:{self.reserve_x}\n"+
                     f"Reserve Y:{self.reserve_y}\n" +
                     f"Total pool tokens :{self.lp_tokens}")

            return lp_minted
        else: 
            print(f"ERROR: incorrect ratio of quantity x and y submitted")
    
    def withdraw_liquidity(self, lp_burned):
        """
        Allows agents to 'burn' liquidity tokens and receive tokens X and Y in return. 
        See 1.D "Remove Liquidity" for an explanation of math. 
        """
        total_lp_tokens = self.lp_tokens
        reserve_x = self.reserve_x
        reserve_y = self.reserve_y 
        
        if self.debug:
            print(f"-"*10, f"WITHDRAW LIQUIDITY ",f"-"*10)
            print(f"-"*10, f"PRIOR LIQUIDITY WITHDRAWN ",f"-"*10)
            print(f"Reserve X:{self.reserve_x}\n"+
                 f"Reserve Y:{self.reserve_y}\n" +
                 f"Total pool tokens :{self.lp_tokens}")

        #calculate tokens to be returned 
        quantity_x = (lp_burned/total_lp_tokens)*reserve_x
        quantity_y = (lp_burned/total_lp_tokens)*reserve_y
        
        #update state 
        self.reserve_x = self.reserve_x - quantity_x
        self.reserve_y = self.reserve_y - quantity_y 
        self.lp_tokens = self.lp_tokens - lp_burned
        
        if self.debug:
            print(f"-"*10, f"AFTER LIQUIDITY WITHDRAWN",f"-"*10)
            print(f"Reserve X:{self.reserve_x}\n"+
                  f"Reserve Y:{self.reserve_y}\n" +
                  f"Total pool tokens :{self.lp_tokens}")

        return_amt_dict = {"quantity_x": quantity_x, "quantity_y": quantity_y}
        
        return return_amt_dict
    
    def request_price(self, transaction_type = "buy",token = None, quantity = 1):
        """
        This method provides you with a price you would have to pay if you want 
        to buy a certain quantity of tokens. However, if you want to sell a certain
        quantity of tokens it offers you a price the institution would pay you. 
        
        Details about this math can be found in the introduction of section 6, 
        plus in 2.B "Buy Tokens" and 2.C "Sell Tokens."
        """
        
        if transaction_type == "buy" : 
            if token == 'x': 
                
                gamma = 1 - self.transaction_fee 
                delta_x = quantity
                new_reserve_x = self.reserve_x - delta_x
                reserve_y = self.reserve_y

                k = self.constant_product
                

                price = (k/(new_reserve_x) - reserve_y)/gamma

                return price

            elif token == 'y':
                
                gamma = 1 - self.transaction_fee 
                delta_y = quantity
                new_reserve_y = self.reserve_y - delta_y
                reserve_x = self.reserve_x

                k = self.constant_product
                

                price = (k/(new_reserve_y) - reserve_x)/gamma

                return price 

            else :
                print(f"ERROR, token = {token} is not traded in this pool")
        
        elif transaction_type == "sell":
            if token == 'x':
                
                gamma = 1 - self.transaction_fee
                delta_x = quantity 
                new_reserve_x = self.reserve_x + delta_x*gamma
                reserve_y = self.reserve_y

                k = self.constant_product
                
                price = reserve_y - k/(new_reserve_x) 
                
                return price
            
            elif token == 'y': 
                
                gamma = 1 - self.transaction_fee
                delta_y = quantity 
                new_reserve_y = self.reserve_y + delta_y*gamma
                reserve_x = self.reserve_x

                k = self.constant_product
                
                price = reserve_x - k/(new_reserve_y) 
                
                return price
            else:
                print(f"ERROR, token = {token} is not traded in this pool")
        else:
            print(f"ERROR, wrong transaction type requested, transaction_type = {transaction_type}")

     
    def sell_tokens(self, token = None, quantity = 0): 
        """
        Allows agents to sell a fixed amount of token X or Y, and 
        receive a quantity of the opposite token in exchange. 
        Then, the institution updates its reserves in the liquidity pool accordingly. 
        See Section 2.C above, "Sell Tokens," for a description.
        """
        if token == 'x': 
            
            x_offered = quantity
            y_returned = self.request_price("sell",token,x_offered)
            
            if self.debug:
                print(f"-"*10, f"SELL TOKEN = {token}", f"-"*10)
                print(f" "*10, f"BEFORE TRADE", f" "*10)
                print(f"Reserve X:{self.reserve_x}\n"+
                      f"Reserve Y:{self.reserve_y}\n" +
                      f"k : {self.constant_product}\n")

            self.reserve_x = self.reserve_x + x_offered
            self.reserve_y = self.reserve_y - y_returned
            
            self.constant_product = self.reserve_x*self.reserve_y
            
            if self.debug:
                print(f" "*10, f"AFTER TRADE", f" "*10)
                print(f"Reserve X:{self.reserve_x}\n"+
                      f"Reserve Y:{self.reserve_y}\n" +
                      f"k : {self.constant_product}\n")

            returned_dict = {"token_returned": "y", "quantity_returned": y_returned}
            
            return returned_dict
        
        elif token == 'y': 
            
            y_offered = quantity
            x_returned = self.request_price("sell",token,y_offered)
            
            if self.debug:
                print(f"-"*10, f"SELL TOKEN = {token}", f"-"*10)
                print(f" "*10, f"BEFORE TRADE", f" "*10)
                print(f"Reserve X:{self.reserve_x}\n"+
                      f"Reserve Y:{self.reserve_y}\n" +
                      f"k : {self.constant_product}\n")

            self.reserve_x = self.reserve_x - x_returned
            self.reserve_y = self.reserve_y + y_offered            
            self.constant_product = self.reserve_x*self.reserve_y
            
            if self.debug:
                print(f" "*10, f"AFTER TRADE", f" "*10)
                print(f"Reserve X:{self.reserve_x}\n"+
                      f"Reserve Y:{self.reserve_y}\n" +
                      f"k : {self.constant_product}\n")

            returned_dict = {"token_returned": "x", "quantity_returned": x_returned}
            
            
            return returned_dict
        else:
            print(f"ERROR, token = {token} is not traded in this pool")
    
    def sell_tokens_min_price(self, token = None, quantity = 0, min_price  = 0 ): 
        """
        Allows agents to sell a fixed amount of token X or Y, and 
        receive a quantity of the opposite token in exchange.
        The Agent is also allowed to specify a min price and the trade only executese if the
        min price condition is met. 
        Then, the institution updates its reserves in the liquidity pool accordingly. 
        See Section 2.C above, "Sell Tokens," for a description.
        """
        if token == 'x': 
            
            x_offered = quantity
            y_returned = self.request_price("sell",token,x_offered)
            
            if y_returned >= min_price: 
                if self.debug:
                    print(f"-"*10, f"SELL TOKEN = {token}", f"-"*10)
                    print(f" "*10, f"BEFORE TRADE", f" "*10)
                    print(f"Reserve X:{self.reserve_x}\n"+
                          f"Reserve Y:{self.reserve_y}\n" +
                          f"k : {self.constant_product}\n")

                self.reserve_x = self.reserve_x + x_offered
                self.reserve_y = self.reserve_y - y_returned

                self.constant_product = self.reserve_x*self.reserve_y

                if self.debug:
                    print(f" "*10, f"AFTER TRADE", f" "*10)
                    print(f"Reserve X:{self.reserve_x}\n"+
                          f"Reserve Y:{self.reserve_y}\n" +
                          f"k : {self.constant_product}\n")

                returned_dict = {"token_returned": "y", "quantity_returned": y_returned}

                return returned_dict
            
            else : 
                
                print(f"-"*10, f"SELL TOKEN = {token}, FAILED", f"-"*10)
                print(f"y_returned  < min_price ")
                print(f"y_returned = {y_returned}" 
                      f"min_price = {min_price}")
                return {"token_returned": "y" , "quantity_returned": "NoTrade"}
                
                
        
        elif token == 'y': 
    
            y_offered = quantity
            x_returned = self.request_price("sell",token,y_offered)
            
            if y_returned >= min_price: 
            
                if self.debug:
                    print(f"-"*10, f"SELL TOKEN = {token}", f"-"*10)
                    print(f" "*10, f"BEFORE TRADE", f" "*10)
                    print(f"Reserve X:{self.reserve_x}\n"+
                          f"Reserve Y:{self.reserve_y}\n" +
                          f"k : {self.constant_product}\n")

                self.reserve_x = self.reserve_x - x_returned
                self.reserve_y = self.reserve_y + y_offered            
                self.constant_product = self.reserve_x*self.reserve_y

                if self.debug:
                    print(f" "*10, f"AFTER TRADE", f" "*10)
                    print(f"Reserve X:{self.reserve_x}\n"+
                          f"Reserve Y:{self.reserve_y}\n" +
                          f"k : {self.constant_product}\n")

                returned_dict = {"token_returned": "x", "quantity_returned": x_returned}


                return returned_dict
            else :
                print(f"-"*10, f"SELL TOKEN = {token}, FAILED", f"-"*10)
                print(f"x_returned  < min_price ")
                print(f"x_returned = {x_returned}" 
                      f"min_price = {min_price}")
                return {"token_returned": "x" , "quantity_returned": "NoTrade"}
                
        else:
            print(f"ERROR, token = {token} is not traded in this pool")
        
    def buy_tokens(self, token = None, quantity = 0):
        """
        Allows agents to request a fixed amount of token X or Y to buy.
        This method returns the price that agent will have to pay, denominated
        in the opposite token. 
        Then, the institution updates its reserves in the liquidity pool accordingly. 
        See Section 2.C above, "Buy Tokens," for a description.
        """
        if token == 'x': 
            
            x_requested = quantity
            y_needed = self.request_price("buy",token, x_requested)
            
            if self.debug:
                print(f"-"*10, f"BUY TOKEN = {token}", f"-"*10)
                print(f" "*10, f"BEFORE TRADE", f" "*10)
                print(f"Reserve X:{self.reserve_x}\n"+
                      f"Reserve Y:{self.reserve_y}\n" +
                      f"k : {self.constant_product}\n")

            self.reserve_x = self.reserve_x - x_requested
            self.reserve_y = self.reserve_y + y_needed
                    
            self.constant_product = self.reserve_x*self.reserve_y
            
            if self.debug:
                print(f" "*10, f"AFTER TRADE", f" "*10)
                print(f"Reserve X:{self.reserve_x}\n"+
                      f"Reserve Y:{self.reserve_y}\n" +
                      f"k : {self.constant_product}\n")

            returned_dict = {"token_needed": "y", "quantity_needed": y_needed}
            
            return returned_dict
        
        elif token == 'y': 
            
            y_requested = quantity
            x_needed = self.request_price("sell",token,y_requested)
            
            if self.debug:
                print(f"-"*10, f"BUY TOKEN = {token}", f"-"*10)
                print(f" "*10, f"BEFORE TRADE", f" "*10)
                print(f"Reserve X:{self.reserve_x}\n"+
                      f"Reserve Y:{self.reserve_y}\n" +
                      f"k : {self.constant_product}\n")

            self.reserve_x = self.reserve_x + x_needed
            self.reserve_y = self.reserve_y - y_requested
        
            self.constant_product = self.reserve_x*self.reserve_y
            
            if self.debug:
                print(f" "*10, f"AFTER TRADE", f" "*10)
                print(f"Reserve X:{self.reserve_x}\n"+
                      f"Reserve Y:{self.reserve_y}\n" +
                      f"k : {self.constant_product}\n")

            returned_dict = {"token_needed": "x", "quantity_needed": x_needed}
            
            return returned_dict
        
        else:
            print(f"ERROR, token = {token} is not traded in this pool")
            
            
    def buy_tokens_max_price(self, token = None, quantity = 0, max_price=0 ):
        """
        Allows agents to request a fixed amount of token X or Y to buy.
        This method returns the price that agent will have to pay, denominated
        in the opposite token. 
        The swap only proceeds if the price required is lower than a specified max_price parameter. 
        Then, the institution updates its reserves in the liquidity pool accordingly. 
        See Section 2.C above, "Buy Tokens," for a description.
        """
        if token == 'x': 
            
            x_requested = quantity
            y_needed = self.request_price("buy",token, x_requested)
            
            if y_needed <= max_price: 
            
                if self.debug:
                    print(f"-"*10, f"BUY TOKEN = {token}", f"-"*10)
                    print(f" "*10, f"BEFORE TRADE", f" "*10)
                    print(f"Reserve X:{self.reserve_x}\n"+
                          f"Reserve Y:{self.reserve_y}\n" +
                          f"k : {self.constant_product}\n")

                self.reserve_x = self.reserve_x - x_requested
                self.reserve_y = self.reserve_y + y_needed

                self.constant_product = self.reserve_x*self.reserve_y

                if self.debug:
                    print(f" "*10, f"AFTER TRADE", f" "*10)
                    print(f"Reserve X:{self.reserve_x}\n"+
                          f"Reserve Y:{self.reserve_y}\n" +
                          f"k : {self.constant_product}\n")

                returned_dict = {"token_needed": "y", "quantity_needed": y_needed}

                return returned_dict
            
            else : 
                print(f"-"*10, f"BUY TOKEN = {token}, FAILED", f"-"*10)
                print(f"y_needed  > max_price ")
                print(f"y_needed = {y_needed}" 
                      f"max_price = {max_price}")
                return {"token_needed": "y" , "quantity_needed": "NoTrade"}

        elif token == 'y': 
            
            y_requested = quantity
            x_needed = self.request_price("sell",token,y_requested)
            
            if x_needed <= max_price: 
            
                if self.debug:
                    print(f"-"*10, f"BUY TOKEN = {token}", f"-"*10)
                    print(f" "*10, f"BEFORE TRADE", f" "*10)
                    print(f"Reserve X:{self.reserve_x}\n"+
                          f"Reserve Y:{self.reserve_y}\n" +
                          f"k : {self.constant_product}\n")

                self.reserve_x = self.reserve_x + x_needed
                self.reserve_y = self.reserve_y - y_requested

                self.constant_product = self.reserve_x*self.reserve_y

                if self.debug:
                    print(f" "*10, f"AFTER TRADE", f" "*10)
                    print(f"Reserve X:{self.reserve_x}\n"+
                          f"Reserve Y:{self.reserve_y}\n" +
                          f"k : {self.constant_product}\n")

                returned_dict = {"token_needed": "x", "quantity_needed": x_needed}

                return returned_dict
            
            else : 
                print(f"-"*10, f"BUY TOKEN = {token}, FAILED", f"-"*10)
                print(f"x_needed  > max_price ")
                print(f"x_needed = {x_needed}" 
                      f"max_price = {max_price}")
                return {"token_needed": "x" , "quantity_needed": "NoTrade"}

        else:
            print(f"ERROR, token = {token} is not traded in this pool")
            
    def provide_liquidity_min_amount(self,amount_x_desired,amount_y_desired, amount_x_min = 0, amount_y_min = 0):
        """
        Provides utility based on certain conditions
        
        """
        if self.debug:
            print(f"-"*10, f"PROVIDE LIQUIDITY ",f"-"*10)
            print(f"-"*10, f"PRIOR LIQUIDITY ",f"-"*10)
            print(f"Reserve X:{self.reserve_x}\n"+
                 f"Reserve Y:{self.reserve_y}\n" +
                 f"Total pool tokens :{self.lp_tokens}")
            print(f"AGENT PARAMS")
            print(f"amount_x_desired = {amount_x_desired}")
            print(f"amount_y_desired = {amount_y_desired}")
            print(f"amount_x_min = {amount_x_min}")
            print(f"amount_y_min = {amount_y_min}")
        
        
        
        amount_y_optimal = amount_x_desired * (self.reserve_y / self.reserve_x)
        if amount_y_optimal <= amount_y_desired: 
            if amount_y_optimal >= amount_y_min:
                amount_x = amount_x_desired
                amount_y = amount_y_optimal
                
                lp_minted = (amount_x/self.reserve_x)*self.lp_tokens
            
                self.lp_tokens = self.lp_tokens + lp_minted 
                self.reserve_x = self.reserve_x + amount_x
                self.reserve_y = self.reserve_y + amount_y
            
                if self.debug:
                    print(f"-"*10, f"AFTER LIQUIDITY INSERTED",f"-"*10)
                    print(f"Reserve X:{self.reserve_x}\n"+
                         f"Reserve Y:{self.reserve_y}\n" +
                         f"Total pool tokens :{self.lp_tokens}\n"+
                         f"amount_x = {amount_x}\n"+
                         f"amount_y = {amount_y}\n")

                return lp_minted
        
            else: 
                print("Insufficient Y Amount")
                print(f"amount_y_optimal < amount_y_min")
                print(f"amount_y_optimal =  {amount_y_optimal}")
                print(f"amount_y_min =  {amount_y_min}")

        else: 
            amount_x_optimal = amount_y_desired * (self.reserve_x / self.reserve_y)
            if amount_x_optimal <= amount_x_desired:
                if amount_x_optimal >= amount_x_min: 
                    amount_x = amount_x_optimal
                    amount_y = amount_y_desired
                    
                    lp_minted = (amount_x/self.reserve_x)*self.lp_tokens
            
                    self.lp_tokens = self.lp_tokens + lp_minted 
                    self.reserve_x = self.reserve_x + amount_x
                    self.reserve_y = self.reserve_y + amount_y

                    if self.debug:
                        print(f"-"*10, f"AFTER LIQUIDITY INSERTED",f"-"*10)
                        print(f"amount_x = ")
                        print(f"Reserve X:{self.reserve_x}\n"+
                             f"Reserve Y:{self.reserve_y}\n" +
                             f"Total pool tokens :{self.lp_tokens}\n"+
                             f"amount_x = {amount_x}\n"+
                             f"amount_y = {amount_y}\n")

                    return lp_minted
             
                else: 
                    print("Insufficient X Amount")
                    print(f"amount_x_optimal < amount_x_min")
                    print(f"amount_x_optimal =  {amount_x_optimal}")
                    print(f"amount_x_min =  {amount_x_min}")

         
        if self.debug:
            print(f"-"*10, f"PROVIDE LIQUIDITY ",f"-"*10)
            print(f"-"*10, f"PRIOR LIQUIDITY ",f"-"*10)
            print(f"Reserve X:{self.reserve_x}\n"+
                 f"Reserve Y:{self.reserve_y}\n" +
                 f"Total pool tokens :{self.lp_tokens}")
        
            
    def set_transaction_fee(self, transaction_fee):
        """
        sets the transaction fee of the uniswap institution to the specified amount
        """
        self.transaction_fee = transaction_fee
        
        

    



In [ ]:

class AMM(object):
    """
    In the simple model below, 
    x tokens represent Energy tokens or E_tokens
    y tokens represent Money tokens or M_tokens 
    """
    def __init__(self,transaction_fee = 0, ratio_error_tol = 1e-7, debug=False):
        
        self.reserve_x = 0
        self.reserve_y = 0
        
        self.constant_product = 0
        self.governance_tokens = 0
        self.lp_tokens = 0 
        
        self.transaction_fee = transaction_fee
        
        self.debug = debug
        self.ratio_error_tol = ratio_error_tol
        
    def setup_pool(self, quantity_x=0, quantity_y=0):
        """
        Establishes a liquidity pool with constant_product = quantity_x * quantity_y.
        Returns liqudity tokens to the agent who sets it up. 
        See 1.A "Establish Liquidity" above for explanation of math. 
        """
        self.reserve_x  = quantity_x
        self.reserve_y = quantity_y
        
        self.constant_product = self.reserve_x*self.reserve_y
        
        lp_minted  = np.sqrt(self.constant_product)
        self.lp_tokens = lp_minted
        
        return lp_minted 
        
    def request_info(self):
        """
        Simple method that informs an agent of the current reserve amounts of 
        X and Y in the liquidity pool. 
        See 1.B "Get Current Amount of Reserves" above for explanation of math. 
        """
        info_dict ={"reserve_x": self.reserve_x, "reserve_y": self.reserve_y,
                       "transaction_fee": self.transaction_fee}
        return info_dict
    
    def provide_liquidity(self,quantity_x, quantity_y): 
        """
        Allows agents to add tokens X and Y to the liquidity pool. 
        Agents must submit X and Y in the exact ratio of the current reserves. 
        Otherwise, method returns an error. 
        If successful, agents receive liquidity tokens in response. 
        See 1.C "Add Liquidity" for an explanation of math. 
        """
        ratio_submitted = quantity_x/quantity_y 
        current_reserve_ratio = self.reserve_x/self.reserve_y
        
        
        if self.debug:
            print(f"-"*10, f"PROVIDE LIQUIDITY ",f"-"*10)
            print(f"-"*10, f"PRIOR LIQUIDITY ",f"-"*10)
            print(f"Reserve X:{self.reserve_x}\n"+
                 f"Reserve Y:{self.reserve_y}\n" +
                 f"Total pool tokens :{self.lp_tokens}")
        
        
        if abs(ratio_submitted - current_reserve_ratio) < self.ratio_error_tol:
            
            lp_minted = (quantity_x/self.reserve_x)*self.lp_tokens
            
            self.lp_tokens = self.lp_tokens + lp_minted 
            self.reserve_x = self.reserve_x + quantity_x 
            self.reserve_y = self.reserve_y + quantity_y 
            
            if self.debug:
                print(f"-"*10, f"AFTER LIQUIDITY INSERTED",f"-"*10)
                print(f"Reserve X:{self.reserve_x}\n"+
                     f"Reserve Y:{self.reserve_y}\n" +
                     f"Total pool tokens :{self.lp_tokens}")

            return lp_minted
        else: 
            print(f"ERROR: incorrect ratio of quantity x and y submitted")
    
    def withdraw_liquidity(self, lp_burned):
        """
        Allows agents to 'burn' liquidity tokens and receive tokens X and Y in return. 
        See 1.D "Remove Liquidity" for an explanation of math. 
        """
        total_lp_tokens = self.lp_tokens
        reserve_x = self.reserve_x
        reserve_y = self.reserve_y 
        
        if self.debug:
            print(f"-"*10, f"WITHDRAW LIQUIDITY ",f"-"*10)
            print(f"-"*10, f"PRIOR LIQUIDITY WITHDRAWN ",f"-"*10)
            print(f"Reserve X:{self.reserve_x}\n"+
                 f"Reserve Y:{self.reserve_y}\n" +
                 f"Total pool tokens :{self.lp_tokens}")

        #calculate tokens to be returned 
        quantity_x = (lp_burned/total_lp_tokens)*reserve_x
        quantity_y = (lp_burned/total_lp_tokens)*reserve_y
        
        #update state 
        self.reserve_x = self.reserve_x - quantity_x
        self.reserve_y = self.reserve_y - quantity_y 
        self.lp_tokens = self.lp_tokens - lp_burned
        
        if self.debug:
            print(f"-"*10, f"AFTER LIQUIDITY WITHDRAWN",f"-"*10)
            print(f"Reserve X:{self.reserve_x}\n"+
                  f"Reserve Y:{self.reserve_y}\n" +
                  f"Total pool tokens :{self.lp_tokens}")

        return_amt_dict = {"quantity_x": quantity_x, "quantity_y": quantity_y}
        
        return return_amt_dict
    
    def request_price(self, transaction_type = "buy",token = None, quantity = 1):
        """
        This method provides you with a price you would have to pay if you want 
        to buy a certain quantity of tokens. However, if you want to sell a certain
        quantity of tokens it offers you a price the institution would pay you. 
        
        Details about this math can be found in the introduction of section 6, 
        plus in 2.B "Buy Tokens" and 2.C "Sell Tokens."
        """
        
        if transaction_type == "buy" : 
            if token == 'x': 
                
                gamma = 1 - self.transaction_fee 
                delta_x = quantity
                new_reserve_x = self.reserve_x - delta_x
                reserve_y = self.reserve_y

                k = self.constant_product
                

                price = (k/(new_reserve_x) - reserve_y)/gamma

                return price

            elif token == 'y':
                
                gamma = 1 - self.transaction_fee 
                delta_y = quantity
                new_reserve_y = self.reserve_y - delta_y
                reserve_x = self.reserve_x

                k = self.constant_product
                

                price = (k/(new_reserve_y) - reserve_x)/gamma

                return price 

            else :
                print(f"ERROR, token = {token} is not traded in this pool")
        
        elif transaction_type == "sell":
            if token == 'x':
                
                gamma = 1 - self.transaction_fee
                delta_x = quantity 
                new_reserve_x = self.reserve_x + delta_x*gamma
                reserve_y = self.reserve_y

                k = self.constant_product
                
                price = reserve_y - k/(new_reserve_x) 
                
                return price
            
            elif token == 'y': 
                
                gamma = 1 - self.transaction_fee
                delta_y = quantity 
                new_reserve_y = self.reserve_y + delta_y*gamma
                reserve_x = self.reserve_x

                k = self.constant_product
                
                price = reserve_x - k/(new_reserve_y) 
                
                return price
            else:
                print(f"ERROR, token = {token} is not traded in this pool")
        else:
            print(f"ERROR, wrong transaction type requested, transaction_type = {transaction_type}")

     
    def sell_tokens(self, token = None, quantity = 0): 
        """
        Allows agents to sell a fixed amount of token X or Y, and 
        receive a quantity of the opposite token in exchange. 
        Then, the institution updates its reserves in the liquidity pool accordingly. 
        See Section 2.C above, "Sell Tokens," for a description.
        """
        if token == 'x': 
            
            x_offered = quantity
            y_returned = self.request_price("sell",token,x_offered)
            
            if self.debug:
                print(f"-"*10, f"SELL TOKEN = {token}", f"-"*10)
                print(f" "*10, f"BEFORE TRADE", f" "*10)
                print(f"Reserve X:{self.reserve_x}\n"+
                      f"Reserve Y:{self.reserve_y}\n" +
                      f"k : {self.constant_product}\n")

            self.reserve_x = self.reserve_x + x_offered
            self.reserve_y = self.reserve_y - y_returned
            
            self.constant_product = self.reserve_x*self.reserve_y
            
            if self.debug:
                print(f" "*10, f"AFTER TRADE", f" "*10)
                print(f"Reserve X:{self.reserve_x}\n"+
                      f"Reserve Y:{self.reserve_y}\n" +
                      f"k : {self.constant_product}\n")

            returned_dict = {"token_returned": "y", "quantity_returned": y_returned}
            
            return returned_dict
        
        elif token == 'y': 
            
            y_offered = quantity
            x_returned = self.request_price("sell",token,y_offered)
            
            if self.debug:
                print(f"-"*10, f"SELL TOKEN = {token}", f"-"*10)
                print(f" "*10, f"BEFORE TRADE", f" "*10)
                print(f"Reserve X:{self.reserve_x}\n"+
                      f"Reserve Y:{self.reserve_y}\n" +
                      f"k : {self.constant_product}\n")

            self.reserve_x = self.reserve_x - x_returned
            self.reserve_y = self.reserve_y + y_offered            
            self.constant_product = self.reserve_x*self.reserve_y
            
            if self.debug:
                print(f" "*10, f"AFTER TRADE", f" "*10)
                print(f"Reserve X:{self.reserve_x}\n"+
                      f"Reserve Y:{self.reserve_y}\n" +
                      f"k : {self.constant_product}\n")

            returned_dict = {"token_returned": "x", "quantity_returned": x_returned}
            
            
            return returned_dict
        else:
            print(f"ERROR, token = {token} is not traded in this pool")
    def sell_tokens_min_price(self, token = None, quantity = 0, min_price  = 0, price_tol=1e-5): 
        """
        Allows agents to sell a fixed amount of token X or Y, and 
        receive a quantity of the opposite token in exchange.
        The Agent is also allowed to specify a min price and the trade only executes if the
        min price condition is met (within error tolerance).
        Then, the institution updates its reserves in the liquidity pool accordingly. 
        See Section 2.C above, "Sell Tokens," for a description.
        """
        if token == 'x': 
            x_offered = quantity
            y_returned = self.request_price("sell", token, x_offered)
            if y_returned + price_tol >= min_price: 
                if self.debug:
                    print(f"-"*10, f"SELL TOKEN = {token}", f"-"*10)
                    print(f" "*10, f"BEFORE TRADE", f" "*10)
                    print(f"Reserve X:{self.reserve_x}\n"+
                          f"Reserve Y:{self.reserve_y}\n" +
                          f"k : {self.constant_product}\n")
                self.reserve_x = self.reserve_x + x_offered
                self.reserve_y = self.reserve_y - y_returned
                self.constant_product = self.reserve_x * self.reserve_y
                if self.debug:
                    print(f" "*10, f"AFTER TRADE", f" "*10)
                    print(f"Reserve X:{self.reserve_x}\n"+
                          f"Reserve Y:{self.reserve_y}\n" +
                          f"k : {self.constant_product}\n")
                returned_dict = {"token_returned": "y", "quantity_returned": y_returned}
                return returned_dict
            else: 
                print(f"-"*10, f"SELL TOKEN = {token}, FAILED", f"-"*10)
                print(f"y_returned < min_price (within tolerance)")
                print(f"y_returned = {y_returned}, min_price = {min_price}, tol = {price_tol}")
                return {"token_returned": "y", "quantity_returned": "NoTrade"}
        elif token == 'y': 
            y_offered = quantity
            x_returned = self.request_price("sell", token, y_offered)
            if x_returned + price_tol >= min_price: 
                if self.debug:
                    print(f"-"*10, f"SELL TOKEN = {token}", f"-"*10)
                    print(f" "*10, f"BEFORE TRADE", f" "*10)
                    print(f"Reserve X:{self.reserve_x}\n"+
                          f"Reserve Y:{self.reserve_y}\n" +
                          f"k : {self.constant_product}\n")
                self.reserve_x = self.reserve_x - x_returned
                self.reserve_y = self.reserve_y + y_offered            
                self.constant_product = self.reserve_x * self.reserve_y
                if self.debug:
                    print(f" "*10, f"AFTER TRADE", f" "*10)
                    print(f"Reserve X:{self.reserve_x}\n"+
                          f"Reserve Y:{self.reserve_y}\n" +
                          f"k : {self.constant_product}\n")
                returned_dict = {"token_returned": "x", "quantity_returned": x_returned}
                return returned_dict
            else:
                print(f"-"*10, f"SELL TOKEN = {token}, FAILED", f"-"*10)
                print(f"x_returned < min_price (within tolerance)")
                print(f"x_returned = {x_returned}, min_price = {min_price}, tol = {price_tol}")
                return {"token_returned": "x", "quantity_returned": "NoTrade"}
        else:
            print(f"ERROR, token = {token} is not traded in this pool")
    def buy_tokens(self, token = None, quantity = 0):
        """
        Allows agents to request a fixed amount of token X or Y to buy.
        This method returns the price that agent will have to pay, denominated
        in the opposite token. 
        Then, the institution updates its reserves in the liquidity pool accordingly. 
        See Section 2.C above, "Buy Tokens," for a description.
        """
        if token == 'x': 
            
            x_requested = quantity
            y_needed = self.request_price("buy",token, x_requested)
            
            if self.debug:
                print(f"-"*10, f"BUY TOKEN = {token}", f"-"*10)
                print(f" "*10, f"BEFORE TRADE", f" "*10)
                print(f"Reserve X:{self.reserve_x}\n"+
                      f"Reserve Y:{self.reserve_y}\n" +
                      f"k : {self.constant_product}\n")

            self.reserve_x = self.reserve_x - x_requested
            self.reserve_y = self.reserve_y + y_needed
                    
            self.constant_product = self.reserve_x*self.reserve_y
            
            if self.debug:
                print(f" "*10, f"AFTER TRADE", f" "*10)
                print(f"Reserve X:{self.reserve_x}\n"+
                      f"Reserve Y:{self.reserve_y}\n" +
                      f"k : {self.constant_product}\n")

            returned_dict = {"token_needed": "y", "quantity_needed": y_needed}
            
            return returned_dict
        
        elif token == 'y': 
            
            y_requested = quantity
            x_needed = self.request_price("sell",token,y_requested)
            
            if self.debug:
                print(f"-"*10, f"BUY TOKEN = {token}", f"-"*10)
                print(f" "*10, f"BEFORE TRADE", f" "*10)
                print(f"Reserve X:{self.reserve_x}\n"+
                      f"Reserve Y:{self.reserve_y}\n" +
                      f"k : {self.constant_product}\n")

            self.reserve_x = self.reserve_x + x_needed
            self.reserve_y = self.reserve_y - y_requested
        
            self.constant_product = self.reserve_x*self.reserve_y
            
            if self.debug:
                print(f" "*10, f"AFTER TRADE", f" "*10)
                print(f"Reserve X:{self.reserve_x}\n"+
                      f"Reserve Y:{self.reserve_y}\n" +
                      f"k : {self.constant_product}\n")

            returned_dict = {"token_needed": "x", "quantity_needed": x_needed}
            
            return returned_dict
        
        else:
            print(f"ERROR, token = {token} is not traded in this pool")
              
    def buy_tokens_max_price(self, token = None, quantity = 0, max_price=0, price_tol=1e-5 ):
        """
        Allows agents to request a fixed amount of token X or Y to buy.
        This method returns the price that agent will have to pay, denominated
        in the opposite token. 
        The swap only proceeds if the price required is lower than a specified max_price parameter,
        with error tolerance allowed (default: 1e-5).
        Then, the institution updates its reserves in the liquidity pool accordingly. 
        See Section 2.C above, "Buy Tokens," for a description.
        """
        if token == 'x': 
            x_requested = quantity
            y_needed = self.request_price("buy",token, x_requested)
            if y_needed <= max_price + price_tol: 
                if self.debug:
                    print(f"-"*10, f"BUY TOKEN = {token}", f"-"*10)
                    print(f" "*10, f"BEFORE TRADE", f" "*10)
                    print(f"Reserve X:{self.reserve_x}\n"+
                          f"Reserve Y:{self.reserve_y}\n" +
                          f"k : {self.constant_product}\n")
                self.reserve_x = self.reserve_x - x_requested
                self.reserve_y = self.reserve_y + y_needed
                self.constant_product = self.reserve_x*self.reserve_y
                if self.debug:
                    print(f" "*10, f"AFTER TRADE", f" "*10)
                    print(f"Reserve X:{self.reserve_x}\n"+
                          f"Reserve Y:{self.reserve_y}\n" +
                          f"k : {self.constant_product}\n")
                returned_dict = {"token_needed": "y", "quantity_needed": y_needed}
                return returned_dict
            else : 
                print(f"-"*10, f"BUY TOKEN = {token}, FAILED", f"-"*10)
                print(f"y_needed  > max_price + tol ")
                print(f"y_needed = {y_needed}" 
                      f"max_price = {max_price}, tol = {price_tol}")
                return {"token_needed": "y" , "quantity_needed": "NoTrade"}

        elif token == 'y': 
            y_requested = quantity
            x_needed = self.request_price("sell",token,y_requested)
            if x_needed <= max_price + price_tol: 
                if self.debug:
                    print(f"-"*10, f"BUY TOKEN = {token}", f"-"*10)
                    print(f" "*10, f"BEFORE TRADE", f" "*10)
                    print(f"Reserve X:{self.reserve_x}\n"+
                          f"Reserve Y:{self.reserve_y}\n" +
                          f"k : {self.constant_product}\n")
                self.reserve_x = self.reserve_x + x_needed
                self.reserve_y = self.reserve_y - y_requested
                self.constant_product = self.reserve_x*self.reserve_y
                if self.debug:
                    print(f" "*10, f"AFTER TRADE", f" "*10)
                    print(f"Reserve X:{self.reserve_x}\n"+
                          f"Reserve Y:{self.reserve_y}\n" +
                          f"k : {self.constant_product}\n")
                returned_dict = {"token_needed": "x", "quantity_needed": x_needed}
                return returned_dict
            else : 
                print(f"-"*10, f"BUY TOKEN = {token}, FAILED", f"-"*10)
                print(f"x_needed  > max_price + tol ")
                print(f"x_needed = {x_needed}" 
                      f"max_price = {max_price}, tol = {price_tol}")
                return {"token_needed": "x" , "quantity_needed": "NoTrade"}

        else:
            print(f"ERROR, token = {token} is not traded in this pool")
            
    def provide_liquidity_min_amount(self,amount_x_desired,amount_y_desired, amount_x_min = 0, amount_y_min = 0):
        """
        Provides utility based on certain conditions
        
        """
        if self.debug:
            print(f"-"*10, f"PROVIDE LIQUIDITY ",f"-"*10)
            print(f"-"*10, f"PRIOR LIQUIDITY ",f"-"*10)
            print(f"Reserve X:{self.reserve_x}\n"+
                 f"Reserve Y:{self.reserve_y}\n" +
                 f"Total pool tokens :{self.lp_tokens}")
            print(f"AGENT PARAMS")
            print(f"amount_x_desired = {amount_x_desired}")
            print(f"amount_y_desired = {amount_y_desired}")
            print(f"amount_x_min = {amount_x_min}")
            print(f"amount_y_min = {amount_y_min}")
        
        
        
        amount_y_optimal = amount_x_desired * (self.reserve_y / self.reserve_x)
        if amount_y_optimal <= amount_y_desired: 
            if amount_y_optimal >= amount_y_min:
                amount_x = amount_x_desired
                amount_y = amount_y_optimal
                
                lp_minted = (amount_x/self.reserve_x)*self.lp_tokens
            
                self.lp_tokens = self.lp_tokens + lp_minted 
                self.reserve_x = self.reserve_x + amount_x
                self.reserve_y = self.reserve_y + amount_y
            
                if self.debug:
                    print(f"-"*10, f"AFTER LIQUIDITY INSERTED",f"-"*10)
                    print(f"Reserve X:{self.reserve_x}\n"+
                         f"Reserve Y:{self.reserve_y}\n" +
                         f"Total pool tokens :{self.lp_tokens}\n"+
                         f"amount_x = {amount_x}\n"+
                         f"amount_y = {amount_y}\n")

                return lp_minted
        
            else: 
                print("Insufficient Y Amount")
                print(f"amount_y_optimal < amount_y_min")
                print(f"amount_y_optimal =  {amount_y_optimal}")
                print(f"amount_y_min =  {amount_y_min}")

        else: 
            amount_x_optimal = amount_y_desired * (self.reserve_x / self.reserve_y)
            if amount_x_optimal <= amount_x_desired:
                if amount_x_optimal >= amount_x_min: 
                    amount_x = amount_x_optimal
                    amount_y = amount_y_desired
                    
                    lp_minted = (amount_x/self.reserve_x)*self.lp_tokens
            
                    self.lp_tokens = self.lp_tokens + lp_minted 
                    self.reserve_x = self.reserve_x + amount_x
                    self.reserve_y = self.reserve_y + amount_y

                    if self.debug:
                        print(f"-"*10, f"AFTER LIQUIDITY INSERTED",f"-"*10)
                        print(f"amount_x = ")
                        print(f"Reserve X:{self.reserve_x}\n"+
                             f"Reserve Y:{self.reserve_y}\n" +
                             f"Total pool tokens :{self.lp_tokens}\n"+
                             f"amount_x = {amount_x}\n"+
                             f"amount_y = {amount_y}\n")

                    return lp_minted
             
                else: 
                    print("Insufficient X Amount")
                    print(f"amount_x_optimal < amount_x_min")
                    print(f"amount_x_optimal =  {amount_x_optimal}")
                    print(f"amount_x_min =  {amount_x_min}")

         
        if self.debug:
            print(f"-"*10, f"PROVIDE LIQUIDITY ",f"-"*10)
            print(f"-"*10, f"PRIOR LIQUIDITY ",f"-"*10)
            print(f"Reserve X:{self.reserve_x}\n"+
                 f"Reserve Y:{self.reserve_y}\n" +
                 f"Total pool tokens :{self.lp_tokens}")
        
            
    def set_transaction_fee(self, transaction_fee):
        """
        sets the transaction fee of the uniswap institution to the specified amount
        """
        self.transaction_fee = transaction_fee
        
class Demand: 
    def __init__(self, v_d, q_d, agent_type, verbose=False): 
        # state variables 
        self.v_d = v_d # per kwh reservation price 
        self.d = q_d # kwh period demand in the current period
        self.remaining_demand = q_d # tracks how much demand has been bought and sold
        self.agent_type = agent_type
        # welfare 
        self.profit = 0 
        self.sw_arr = []

        # AMM Instance 
        self.AMM = AMM # NOTE: return here

        # txs request 
        self.txs_request = None 
        

        # debugging
        self.verbose = verbose

        print(f"[{self.agent_type}][INIT] v_d={v_d}, q_d={q_d}, AMM={AMM}, verbose={verbose}")

    def update_profit(self, p, q): 
        utils = self.util_func(p, q)
        self.profit += utils
        self.sw_arr.append(utils)
        print(f"[{self.agent_type}][update_profit] Updated profit by {utils}, total profit now {self.profit}")
        if self.verbose:
            print(f"[{self.agent_type}][update_profit][VERBOSE] p={p}, q={q}, utils={utils}, sw_arr={self.sw_arr}")

    
    def util_func(self, p, q):
        result = (self.v_d - p) * q
        if self.verbose:
            print(f"[{self.agent_type}][util_func][VERBOSE] v_d={self.v_d}, p={p}, q={q}, result={result}")
        return result
    
    def make_decision(self, E, M):
        print(f"[{self.agent_type}][make_decision] Called with E={E}, M={M}")
        q_d = self.remaining_demand # tracks the amount of demand not purchased
        print(f"[{self.agent_type}][make_decision] Remaining demand: {q_d}")

        if round(q_d, 12) > 0: # if there is positive demand remaining
            exp_total_amm_price = self.calc_price(E, M, q_d) 
            print(f"[{self.agent_type}][make_decision] Expected AMM price for {q_d}: {exp_total_amm_price}")

            if exp_total_amm_price > (self.v_d * q_d): 
                e_req = E - (M / self.v_d) # NOTE: could be wrong and needs to be tested 
                print(f"[{self.agent_type}][make_decision] AMM price too high, calculating e_req: {e_req}")

                if e_req <= 0:
                    print(f"[{self.agent_type}][make_decision] Not enough liquidity to buy. Skipping trade.")
                    return None

                exp_total_amm_price = self.calc_price(E, M, e_req)
                print(f"[{self.agent_type}][make_decision] New expected AMM price for e_req={e_req}: {exp_total_amm_price}")

                txs_request = {
                    "decision": "buy_tokens_max_price",
                    "token": "x",
                    "quantity": e_req,
                    "max_price": self.v_d * e_req,
                    "agent_type": self.agent_type
                }
                if self.verbose:
                    print(f"[{self.agent_type}][make_decision][VERBOSE] txs_request={txs_request}")
            else: 
                e_req = q_d
                txs_request = {
                    "decision": "buy_tokens_max_price",
                    "token": "x",
                    "quantity": q_d,
                    "max_price": self.v_d * e_req,
                    "agent_type": self.agent_type
                }
                if self.verbose:
                    print(f"[{self.agent_type}][make_decision][VERBOSE] txs_request={txs_request}")

            print(f"[{self.agent_type}][make_decision] Final txs_request: {txs_request}")
            self.txs_request = txs_request
            return txs_request

        else: 
            print(f"[{self.agent_type}][make_decision] No remaining demand.")
            return 
              

    def calc_price(self, E, M, e): 
        if self.verbose:
            print(f"[{self.agent_type}][calc_price][VERBOSE] E={E}, M={M}, e={e}")
        
        if abs(E - e) < 1e-12:
            print(f"[{self.agent_type}][calc_price] Requesting entire pool, returning inf cost.")
            return float("inf")

        k = E * M
        try:
            m = k / (E - e) - M
        except ZeroDivisionError:
            print(f"[{self.agent_type}][calc_price] Division by zero, returning inf.")
            return float("inf")

        if m < 0:
            print(f"[{self.agent_type}][calc_price] Invalid negative price {m:.4f}. Returning inf.")
            return float("inf")

        if self.verbose:
            print(f"[{self.agent_type}][calc_price][VERBOSE] k={k}, m={m}")

        return m


    def transaction_result(self, txs_result): 
        print(f"[{self.agent_type}][transaction_result] txs_result: {txs_result}")
        e_requested = self.txs_request["quantity"]
        m_required = txs_result["quantity_needed"]
        avg_price = m_required / e_requested

        print(f"[{self.agent_type}][transaction_result] avg_price = {avg_price}")

        self.update_profit(avg_price, e_requested)
        self.remaining_demand -= e_requested
        print(f"[{self.agent_type}][transaction_result] Updated remaining_demand: {self.remaining_demand}")



# class Supply: 
#     def __init__(self, c, q_sell, agent_type, verbose=False): 
#         # state variables 
#         self.c = c # per kwh reservation price 
#         self.q_sell = q_sell # kwh period demand in the current period
#         self.remaining_supply = q_sell # tracks how much demand has been bought and sold

#         # welfare 
#         self.profit = 0 
#         self.sw_arr = []

      
#         # txs request 
#         self.txs_request = None 
#         self.agent_type = agent_type

#         # debugging
#         self.verbose = verbose

#         print(f"[{self.agent_type}][INIT] c={c}, q_sell={q_sell}, AMM={AMM}, verbose={verbose}")

#     def update_profit(self, p, q): 
#         utils = self.util_func(p, q)
#         self.profit += utils
#         self.sw_arr.append(utils)
#         print(f"[{self.agent_type}][update_profit] Updated profit by {utils}, total profit now {self.profit}")
#         if self.verbose:
#             print(f"[{self.agent_type}][update_profit][VERBOSE] p={p}, q={q}, utils={utils}, sw_arr={self.sw_arr}")

#     def make_decision(self, E, M):
#         print(f"[{self.agent_type}][make_decision] Called with E={E}, M={M}")
#         q_s = self.remaining_supply # tracks the amount of supply not purchased
#         print(f"[{self.agent_type}][make_decision] Remaining supply: {q_s}")

#         if round(q_s, 12) > 0: # if there is positive supply remaining
#             exp_total_amm_price = self.calc_price(E, M, q_s)
#             print(f"[{self.agent_type}][make_decision] Expected AMM price for {q_s}: {exp_total_amm_price}")

#             if exp_total_amm_price < (self.c * q_s): 
#                 e_sell = (M / self.c) - E # NOTE: could be wrong and needs to be tested 
#                 print(f"[{self.agent_type}][make_decision] AMM price too low, calculating e_sell: {e_sell}")
#                 exp_total_amm_price = self.calc_price(E, M, e_sell)

#                 if e_sell < 0:
#                     print(f"[{self.agent_type}][make_decision] No profitable trades remaining.")
#                     return # no profitable trades remaining

#                 txs_request = {
#                     "decision": "sell_tokens_min_price",
#                     "token": "x",
#                     "quantity": e_sell,
#                     "min_price": self.c * e_sell,
#                     "agent_type": self.agent_type
#                 }
#                 if self.verbose:
#                     print(f"[{self.agent_type}][make_decision][VERBOSE] txs_request={txs_request}")
#             else: 
#                 e_sell = q_s
#                 txs_request = {
#                     "decision": "sell_tokens_min_price",
#                     "token": "x",
#                     "quantity": e_sell,
#                     "min_price": self.c * e_sell,
#                     "agent_type": self.agent_type
#                 }
#                 if self.verbose:
#                     print(f"[{self.agent_type}][make_decision][VERBOSE] txs_request={txs_request}")

#             print(f"[{self.agent_type}][make_decision] Final txs_request: {txs_request}")
#             self.txs_request = txs_request
#             return txs_request

#         else: 
#             print(f"[{self.agent_type}][make_decision] No remaining supply.")
#             return 
        
#     def transaction_result(self, txs_result): 
#         print(f"[{self.agent_type}][transaction_result] txs_result: {txs_result}")
#         e_sell = self.txs_request["quantity"]
#         m_returned = txs_result["quantity_returned"]
#         avg_price = m_returned / e_sell

#         print(f"[{self.agent_type}][transaction_result] avg_price = {avg_price}, self.c = {self.c}")

#         self.update_profit(avg_price, e_sell)
#         self.remaining_supply -= e_sell
#         print(f"[{self.agent_type}][transaction_result] Updated remaining_supply: {self.remaining_supply}")

#     def util_func(self, p, q):
#         result = (p - self.c) * q
#         if self.verbose:
#             print(f"[{self.agent_type}][util_func][VERBOSE] p={p}, c={self.c}, q={q}, result={result}")
#         return result
    
#     def calc_price(self, E, M, e): 
#         if self.verbose:
#             print(f"[{self.agent_type}][calc_price][VERBOSE] E={E}, M={M}, e={e}")
#         k = E * M 
#         m = M - k / (E + e)
#         if self.verbose:
#             print(f"[{self.agent_type}][calc_price][VERBOSE] k={k}, m={m}")
#         return m


class Model(): 

    def __init__(self, T, agent_list, amm_instance, trades_per_period, verbose=False): 
        self.total_periods = T 
        self.agent_list = agent_list
        self.trades_per_period = trades_per_period
        self.amm_instance = amm_instance
        self.verbose = verbose

        # reserve_x  = self.amm_instance.request_info()["reserve_x"]
        # reserve_y  = self.amm_instance.request_info()["reserve_y"]

        # self.amm_state_list = [(reserve_x,reserve_y)]

        self.amm_period_final_state = [] #logs the state of the amm at the end of the period
    

    def simulate(self): 
        print(f"[Model][simulate] Starting simulation for {self.total_periods} periods, {self.trades_per_period} trades per period.")

        for t in range(self.total_periods): 
            print(f"\n[Model][simulate] === Period {t+1}/{self.total_periods} ===")

            for trade in range(self.trades_per_period): 
                print(f"[Model][simulate] -- Trade {trade+1}/{self.trades_per_period} --")
                random.shuffle(self.agent_list)
                if self.verbose:
                    print(f"[Model][simulate][VERBOSE] Agent list order: {[type(a).__name__ for a in self.agent_list]}")
                for agent in self.agent_list: 
                    print(f"[Model][simulate] Agent: {type(agent).__name__}")
                    # Get the AMM state 
                    amm_state = self.amm_instance.request_info()
                    E = amm_state["reserve_x"]
                    M = amm_state["reserve_y"]
                    print(f"[Model][simulate] AMM State: E={E}, M={M}")
                    if self.verbose:
                        print(f"[Model][simulate][VERBOSE] AMM full state: {amm_state}")

                    # Get the txs 
                    amm_txs_request = agent.make_decision(E, M, t)
                    print(f"[Model][simulate] Agent decision: {amm_txs_request}")
                    if amm_txs_request:
                        decision = amm_txs_request["decision"]
                        if decision == "sell_tokens_min_price":
                            token = amm_txs_request["token"]
                            quantity = amm_txs_request["quantity"]
                            min_price = amm_txs_request["min_price"]
                            print(f"[Model][simulate] Agent requests to SELL {quantity} {token} with min_price {min_price}")

                            txs_result = self.amm_instance.sell_tokens_min_price(token, quantity, min_price)
                            quantity_returned = txs_result["quantity_returned"]
                            print(f"[Model][simulate] AMM sell_tokens_min_price result: {txs_result}")
                            
                            if quantity_returned == "NoTrade": 
                                print(f"[Model][simulate] Trade not executed (NoTrade).")
                                continue 
                            else:
                                print(f"[Model][simulate] Trade executed. Passing result to agent.")
                                agent.transaction_result(txs_result)

                        elif decision == "buy_tokens_max_price":
                            token = amm_txs_request["token"] 
                            quantity = amm_txs_request["quantity"]
                            max_price = amm_txs_request["max_price"]
                            print(f"[Model][simulate] Agent requests to BUY {quantity} {token} with max_price {max_price}")

                            txs_result = self.amm_instance.buy_tokens_max_price(token, quantity, max_price)
                            quantity_needed  = txs_result["quantity_needed"]
                            print(f"[Model][simulate] AMM buy_tokens_max_price result: {txs_result}")

                            if quantity_needed == "NoTrade": 
                                print(f"[Model][simulate] Trade not executed (NoTrade).")
                                continue 
                            else:
                                print(f"[Model][simulate] Trade executed. Passing result to agent.")
                                agent.transaction_result(txs_result)
                        else: 
                            print(f"[Model][simulate] ERROR: Unknown decision type: {decision}")
                            raise Exception("Unknown decision type encountered in Model.simulate()")
                    else:
                        print(f"[Model][simulate] No trade decision from agent.")

            reserve_x  = self.amm_instance.request_info()["reserve_x"]
            reserve_y  = self.amm_instance.request_info()["reserve_y"]

            print(f"reserve_x = {reserve_x}")
            print(f"reserve_y = {reserve_y}")
            tup = (reserve_x,reserve_y)
            self.amm_period_final_state.append(tup)
            
            
        print("[Model][simulate] Simulation complete.")




### Elastic ABM Demand agent 

In [ ]:
class ElasticDemand: 
    def __init__(self, v_max, q_max, agent_type, verbose=False): 
        # state variables 
        self.v_max = v_max # per kwh reservation price 
        self.q_max= q_max # max demand or total demand 
        
        self.agent_type = agent_type

        self.remaining_demand = q_max
        # welfare 
        self.profit = 0 
        self.sw_arr = []

        # txs request 
        self.txs_request = None 
        
        # debugging
        self.verbose = verbose

        #AMM state 
        self.amm_state = {}

        if self.verbose:
            print(f"[{self.agent_type}][INIT] v_max={v_max}, q_max={q_max}, agent_type={agent_type}, verbose={verbose}")

    def value_function(self,q): 
        """ Marginal Value for unit q purchased"""
        return self.v_max*(1- q/self.q_max) 
    
    def util_func(self, q, token_required):
        """ 
        Calculates the utility or consumer surplus from trade
        """

        # area under demand
        surplus = self.v_max * (q - q**2/(2*self.q_max))
        # total paid
        paid = token_required
        consumer_surplus = surplus - paid

        return consumer_surplus

    def find_optimal_e(self, E, M, v_max, q_max):
        """
        Solves for e in:
        e^3 -(q_max + 2E)e^2 + (E^2 + 2E q_max)e + q_max*E*(M/v_max - E) = 0
        Returns the unique real root 0 < e < E.
        """
        # Coefficients of the monic cubic
        coeffs = [
            1,
            -(q_max + 2*E),
            E**2 + 2*E*q_max,
            q_max*E*(M/v_max - E)
        ]
        roots = np.roots(coeffs)
        # Filter for a real root in (0, E)
        real_roots = [r.real for r in roots if abs(r.imag) < 1e-8 and 0 < r.real < E]
        if not real_roots:
            print("No valid root found in (0, E)")
        return real_roots 


    def update_profit(self, q, token_required): 
        utils = self.util_func(q, token_required)
        self.profit += utils
        self.sw_arr.append(utils)
        print(f"[{self.agent_type}][update_profit] Updated profit by {utils}, total profit now {self.profit}")
        if self.verbose:
            print(f"[{self.agent_type}][update_profit][VERBOSE] p={p}, q={q}, utils={utils}, sw_arr={self.sw_arr}")
    
    
    def make_decision(self, E, M,t):
        """Compute the optimal e* and issue a buy_tokens_max_price at that e."""
        if self.remaining_demand <= 0:
            if self.verbose: print(f"[{self.agent_type}] No demand left.")
            return None
        
        #AMM_state 
        self.update_amm_state(E, M)
        e_star = self.find_optimal_e(E, M, self.v_max, self.q_max)
        
        if not e_star: return # if no real roots, we return 

        print(e_star)
        e_star = e_star[0]
        e_req = min(e_star, self.remaining_demand)
        p_star = self.marginal_price(E, M, e_req)

        tx = {
            "decision":   "buy_tokens_max_price",
            "token":      "x",
            "quantity":   e_req,
            "max_price":  p_star * e_req, 
            "agent_type": self.agent_type
        }

        if self.verbose:
            print(f"[{self.agent_type}][make_decision] e*={e_star:.4f}, e_req={e_req:.4f}, p*={p_star:.4f}")
            print(f"[{self.agent_type}][make_decision][VERBOSE] tx = {tx}")

        self.txs_request = tx
        return tx
    
    def update_amm_state(self, E, M): 

        self.amm_state = {"reserve_x" :E, "reserve_y": M}
    def marginal_price(self, E, M, e):
        """Marginal cost MC(e) = d/d e [ total_cost(e) ]."""
        K = E * M
        return K / (E - e)**2

    def transaction_result(self, txs_result): 
        print(f"[{self.agent_type}][transaction_result] txs_result: {txs_result}")
        e_requested = self.txs_request["quantity"]
        m_required = txs_result["quantity_needed"]
        
        #Calculating the Average Price 
        avg_price = m_required/e_requested
        print(f"[{self.agent_type}][transaction_result] avg_price = {avg_price}")
        
        #Figuring out the Marginal value and Cost 
        E , M = self.amm_state["reserve_x"], self.amm_state["reserve_y"]
        marginal_cost = self.marginal_price(E, M, e_requested)
        marginal_value= self.value_function(e_requested)

        print(f"[{self.agent_type}][transaction_result] Marginal Value = {marginal_value}, Marginal Cost = {marginal_cost}")

        self.update_profit(e_requested, m_required)
        self.remaining_demand -= e_requested
        self.q_max -= e_requested 
        self.v_max = self.value_function(e_requested)

        print(f"[{self.agent_type}][transaction_result] Updated remaining_demand: {self.remaining_demand}")

        

## Marginal Cost Supply 

In [ ]:
class Supply:
    def __init__(self, c, q_sell, agent_type, verbose=False):
        # constant per-unit cost, total supply endowment
        self.c = c
        self.q_sell = q_sell
        self.remaining_supply = q_sell

        # welfare tracking
        self.profit = 0.0
        self.sw_arr = []

        # last request
        self.txs_request = None
        self.agent_type = agent_type
        self.verbose = verbose

        #AMM 
        self.amm_state = {}

        print(f"[{self.agent_type}][INIT] c={c}, q_sell={q_sell}, verbose={verbose}")

    def update_amm_state(self, E, M): 

        self.amm_state = {"reserve_x": E, "reserve_y": M}

    def marginal_price(self, E, M, e):
        """AMM marginal revenue when selling e of x into the E-reserve."""
        K = E * M
        return K / (E + e) ** 2

    def find_optimal_e(self, E, M):
        """
        Closed-form optimal sell quantity when MC=c:
          if c>0: solve c = K/(E+e)^2  =>  e = sqrt(K/c) − E
          if c<=0: supply everything up to your remaining endowment
        """
        if self.c <= 0:
            # zero (or negative) marginal cost → willing to sell all you have
            return self.remaining_supply

        K = E * M
        e_star = np.sqrt(K / self.c) - E
        return max(0.0, e_star)

    def util_func(self, p, q):
        """(price – cost) × quantity."""
        return (p - self.c) * q

    def update_profit(self, p, q):
        utils = self.util_func(p, q)
        self.profit += utils
        self.sw_arr.append(utils)
        if self.verbose:
            print(f"[{self.agent_type}][update_profit] profit={utils:.4f}, total={self.profit:.4f}")

    def make_decision(self, E, M, t):
        """Compute e* and issue a sell_tokens_min_price order."""
        self.update_amm_state(E,M)

        if self.remaining_supply <= 0:
            if self.verbose: print(f"[{self.agent_type}] No remaining supply.")
            return None

        e_star = self.find_optimal_e(E, M)
        e_req  = min(e_star, self.remaining_supply)

        if e_req <= 0:
            if self.verbose: print(f"[{self.agent_type}] No profitable quantity to sell.")
            return None

        p_star = self.marginal_price(E, M, e_req)

        if self.c <= 0: p_star = 0 # you are willing to accept any price 

        tx = {
            "decision":   "sell_tokens_min_price",
            "token":      "x",
            "quantity":   e_req,
            # specify total minimum y-tokens you must get back
            "min_price":  p_star * e_req,
            "agent_type": self.agent_type
        }
        if self.verbose:
            print(f"[{self.agent_type}][make_decision] e*={e_star:.4f}, e_req={e_req:.4f}, p*={p_star:.4f}")
            print(f"[{self.agent_type}][make_decision][VERBOSE] tx = {tx}")

        self.txs_request = tx
        return tx

    def transaction_result(self, txs_result):
        """Update profit and remaining_supply after execution."""
        e_sold      = self.txs_request["quantity"]
        m_received  = txs_result["quantity_returned"]
        avg_price   = m_received / e_sold

        #Figuring out Marginal Cost and Marginal Revenue 
        E = self.amm_state["reserve_x"]
        M = self.amm_state["reserve_y"]
        mr_rev = self.marginal_price(E, M, e_sold)
        
        self.update_profit(avg_price, e_sold)
        self.remaining_supply -= e_sold
        if self.verbose:
            print(f"[{self.agent_type}][transaction_result] Marginal Cost = {self.c} Marginal Revenue = {mr_rev}")
            print(f"[{self.agent_type}][transaction_result] remaining = {self.remaining_supply:.4f}")


### Battery Strategy 

In [ ]:

class Battery: 

    def __init__(self, C_init, C_max, agent_type , s_t, v_max, q_max, c_u, q_u_max, verbose = False): 

        self.C_init = C_init # this is the initial and final state of the battery
        self.C_max = C_max 
        self.C = C_init

        self.agent_type = agent_type

        self.verbose = verbose
        
        # welfare tracking
        self.profit = 0.0
        self.sw_arr = []
        self.amm_state = {}
        
        if self.verbose: print(f"[{self.agent_type}][init] ", 
                               f"C_init = {self.C_init}",
                               f"C_max = {self.C_max}",
                               f"agent_type = {self.agent_type}",)
            
        #buyer and seller params to calc equilibrium prices 
        self.s_t = s_t  #array of solar generation in all periods t 
        self.v_max = v_max # max value of demand 
        self.q_max = q_max # max quantity demanded 
        self.c_u = c_u #per unit marginal cost of supply 
        self.q_u_max = q_u_max # max procurement by the utility this should be q_max

    def util_func(self, p, q):
        """price * quantity."""
        return p * q
    
    def update_profit(self, p, q):

        print(f"[{self.agent_type}] p = {p}, {q}")
        utils = self.util_func(p, q)
        self.profit += utils
        self.sw_arr.append(utils)
        if self.verbose:
            print(f"[{self.agent_type}][update_profit] profit={utils:.4f}, total={self.profit:.4f}")

    def update_amm_state(self, E, M): 
        """reserve_x are the energy reserves, and reserve y are the money reserves"""
        self.amm_state = {"reserve_x": E, "reserve_y": M}



    def make_decision(self, E, M, t):
        """
        1) Compute spot p_t and forecast p_{t+1}.
        2) Decide buy/sell/hold and q_req.
        3) Execute via amm.buy_tokens_max_price or amm.sell_tokens_min_price.
        """
        self.update_amm_state(E, M)
        
        # 1) true spot price
        p_t = M / E

        K = E * M
        # 2) forecast next period equilibrium *demand-side* price
        #    you can still use your comp_eq_price to get p_{t+1} from solar+utility stack
        p_nxt, _ = self.comp_eq_price_quantity(t + 1)

        print(f"[{self.agent_type}][make_decision] E={E}, M={M}, t={t}, p_t={p_t}, K={K}, p_nxt={p_nxt}, SOC={self.C}, C_max={self.C_max}")

        if self.verbose:
            print(f"[{self.agent_type}] p_t={p_t:.4f}, p_{t+1}={p_nxt:.4f}, SOC={self.C:.4f}")

        # 1) BUY (charge)
        if (p_nxt > p_t) and (p_nxt > 0):
           
            # correct unconstrained q* for buying E-tokens
            q_uncon = E - np.sqrt(K / p_nxt)

            # clamp to available battery capacity and to not drain pool completely
            q_req = float(np.clip(q_uncon, 0.0, min(self.C_max - self.C, E - 1e-8)))
           
            if q_req <= 0:
                return None

            # worst-case final per-unit cost at e = q_req
            p_final   = K / (E - q_req)**2
            max_total = p_final * q_req

            tx = {
            "decision":   "buy_tokens_max_price",
            "token":      "x",
            "quantity":   q_req,
            "max_price":  max_total,
            "agent_type": self.agent_type
            }
            
            self.txs_request = tx
            return tx

        # 2) SELL (discharge)
        elif p_nxt < p_t:
            if p_nxt > 0: 
                q_uncon = np.sqrt(K / p_nxt) - E
            else: # if next period price is 0 we sell all our holdings. 
                q_uncon = self.C

            print(f"[{self.agent_type}][make_decision][SELL] q_uncon={q_uncon}")

            # clamp into [0, SOC]
            q_req  = float(np.clip(q_uncon, 0.0, self.C))

            print(f"[{self.agent_type}][make_decision][SELL] q_req={q_req}")

            if q_req <= 0:
                print(f"[{self.agent_type}][make_decision][SELL] q_req <= 0, returning None")
                return None
            
            p_final = K / (E + q_req)**2
            min_total = p_final * q_req

            tx = {
            "decision":   "sell_tokens_min_price",
            "token":      "x",
            "quantity":   q_req,
            "min_price":  min_total,
            "agent_type": self.agent_type
            }
            self.txs_request = tx 
            print(f"[{self.agent_type}][make_decision][SELL] tx={tx}")
            return tx

        # 3) HOLD
        else:
            print(f"[{self.agent_type}][make_decision][HOLD] No action taken.")
            if self.verbose:
                print(f"[{self.agent_type}] hold")
            return None
    
    def transaction_result(self, txs_result):
        """Update profit and remaining_supply after execution."""

        print(f"[{self.agent_type}][transaction_result] txs_result: {txs_result}")

        #if your previous decision was to sell tokens
        if self.txs_request["decision"] == "sell_tokens_min_price":
            e_sold      = self.txs_request["quantity"]
            m_received  = txs_result["quantity_returned"]
            avg_price   = m_received / e_sold

            print(f"[{self.agent_type}][transaction_result] e_sold = {e_sold}")
            print(f"[{self.agent_type}][transaction_result] m_received = {m_received}")
            print(f"[{self.agent_type}][transaction_result] avg_price = {avg_price}")

            self.update_profit(avg_price, e_sold)

            #NOTE: could be a bug if e_sold is greater than self.C
            self.C -= e_sold #you update your battery state to reflect amount of e that was discharged

            print(f"[{self.agent_type}][transaction_result] Updated SOC (C) = {self.C}")

        elif self.txs_request["decision"] == "buy_tokens_max_price":
            e_requested = self.txs_request["quantity"]
            m_required = txs_result["quantity_needed"]

            print(f"[{self.agent_type}][transaction_result] e_requested = {e_requested}")
            print(f"[{self.agent_type}][transaction_result] m_required = {m_required}")

            #Calculating the Average Price 
            avg_price = m_required / e_requested

            print(f"[{self.agent_type}][transaction_result] avg_price = {avg_price}")
            
            self.update_profit(-avg_price, e_requested)
            self.C += e_requested 
            self.C = min(self.C_max, self.C) #incase there is an overcharge 

            print(f"[{self.agent_type}][transaction_result] Updated SOC (C) = {self.C}")

        else: 
            raise Exception(f"[{self.agent_type}][transaction_result] Unknown txs_request decision {self.txs_request}")
                

    
    def comp_eq_price_quantity(self, t):
        """
        Find (p*, q*) solving demand v(q) = supply stack:
        - Solar free up to s_t[t]
        - Utility at cost c_u up to q_u_max
        Demand: v(q)=v_max*(1-q/q_max)
        """

        # 1) If solar alone covers full demand at p=0
        if self.q_max <= self.s_t[t]:
            return 0.0, self.q_max

        # 2) If solar < full demand, but the next marginal value is below c_u:
        #    v(s_t) = v_max*(1 - s_t/q_max) < c_u
        #    ⇒ nobody is willing to pay the utility price, so
        #    market clears on free solar alone
        v_at_solar = self.v_max * (1 - self.s_t[t] / self.q_max)
        if v_at_solar < self.c_u:
            return 0.0, self.s_t[t]

        # 3) Otherwise demand at price c_u (utility kicks in)
        q_star = self.q_max * (1 - self.c_u / self.v_max)
        total_supply = self.s_t[t] + self.q_u_max

        # 3a) If that demand exceeds total capacity, clear at choke price
        if q_star > total_supply:
            p_star = self.v_max * (1 - total_supply / self.q_max)
            return p_star, total_supply

        # 3b) Otherwise clear at (c_u, q_star)
        return self.c_u, q_star
    
    # def comp_eq_price_quantity(self, t):
    #     """
    #     Find (p*, q*) solving demand v(q) = supply price curve:
    #     - Solar:    up to s_t at price 0
    #     - Utility:  next q_u_max at price c_u
    #     - Beyond:   no supply (price = +inf)

    #     Demand inverse: v(q) = v_max*(1 - q/q_max)

    #     Returns:
    #     p_star, q_star
    #     """
    #     # 1) If zero‐price solar alone suffices to meet demand at p=0:
    #     #    demand at p=0 is q_d0 = q_max * (1 - 0/v_max) = q_max.
    #     #    If q_max <= s_t, solar meets all demand → p*=0, q*=q_max.
    #     if self.q_max <= self.s_t[t]:
    #         return 0.0, self.q_max

    #     # 2) Otherwise solar alone is not enough, price must jump to c_u.
    #     #    Solve v(q) = c_u  =>  q* = q_max*(1 - c_u/v_max).
    #     q_star = self.q_max * (1 - self.c_u / self.v_max)

    #     # Clamp between what solar+utility can supply:
    #     total_supply = self.s_t[t] + self.q_max
        
    #     if q_star > total_supply:
    #         # Even at utility cost, demand exceeds capacity → market clears
    #         # at the marginal supplier's choke price (infinite), but in practice
    #         # you'd set p* so that q* = total_supply:
    #         #   v(total_supply) = p*
    #         p_star = self.v_max * (1 - total_supply / self.q_max)
    #         return p_star, total_supply

    #     # Otherwise, equilibrium at (c_u, q_star)
    #     return self.c_u, q_star

        
        

In [65]:
import numpy as np

def equilibrium_with_battery_offer(
    s_t: float,
    q_b_offer: float,
    v_max: float,
    q_max: float,
    c_u: float,
    q_u_max: float
):
    """
    Compute equilibrium price/quantities given:
      - solar supply at price 0: s_t
      - battery supply at price 0: q_b_offer
      - utility supply at price c_u: capacity q_u_max
      - linear inverse‐demand v(q)=v_max*(1 - q/q_max)

    Returns:
      p_star, q_d, q_s, q_b, q_u
    """
    # 1) total zero‐price supply
    free_supply = s_t + q_b_offer

    # 2) demand at zero price
    q_d0 = q_max  # v(0)=v_max>0 ⇒ q at p=0 is q_max

    # 3) if demand <= free_supply, clear at p=0
    if q_d0 <= free_supply:
        q_d = q_d0
        p_star = 0.0
        # allocate: solar up to its capacity
        q_s = min(s_t, q_d)
        q_b = max(0.0, q_d - q_s)  # avoid negatives due to precision
        q_u = 0.0
        return p_star, q_d, q_s, q_b, q_u

    # 4) check whether the marginal value at free_supply is below c_u
    #    if so, price remains 0 but quantity = free_supply
    v_at_free = v_max * (1 - free_supply / q_max)
    if v_at_free < c_u:
        q_d = free_supply
        p_star = 0.0
        q_s = s_t
        q_b = q_b_offer
        q_u = 0.0
        return p_star, q_d, q_s, q_b, q_u

    # 5) otherwise bring utility online at cost c_u
    #    demand at p=c_u
    q_d_cu = q_max * (1 - c_u / v_max)
    total_capacity = free_supply + q_u_max

    # 5a) if that demand exceeds total capacity, capacity‐constrained
    if q_d_cu > total_capacity:
        q_d = total_capacity
        p_star = c_u
    else:
        q_d = q_d_cu
        p_star = c_u

    # 6) allocate across sources in dispatch order
    #    solar first, then battery, then utility
    remaining = q_d
    q_s = min(s_t, remaining)
    remaining -= q_s

    q_b = min(q_b_offer, remaining)
    remaining -= q_b

    q_u = min(q_u_max, max(0.0, remaining))  # enforce non-negativity

    # remaining should be ~0
    return p_star, q_d, q_s, q_b, q_u


def equilibrium_with_battery_bid(
    s_t: float,
    q_b_bid: float,
    v_max: float,
    q_max: float,
    c_u: float,
    q_u_max: float
):
    """
    Supply stack:
      1) Solar at p=0, capacity s_t
      2) Utility at p=c_u, capacity q_u_max

    Demand:
      1) Battery bids q_b_bid at p=0 (charges only if price=0 and solar > consumer)
      2) Consumers with inverse‐demand v(q)=v_max*(1 - q/q_max)

    Returns:
      p_star : equilibrium price
      q_d    : consumer quantity
      q_s    : solar dispatched
      q_b    : battery charged (bid filled)
      q_u    : utility dispatched
    """
    # Consumer demand at zero price
    q_d0 = q_max  # since v(q)=0 => q=q_max
    
    # 1) Check if market clears at p=0 on solar alone:
    if s_t >= q_d0:
        # Solar > consumer demand: price=0, consumers get q_max, battery fills from excess
        p_star = 0.0
        q_d = q_d0
        # battery charges only from leftover solar
        q_b = min(q_b_bid, s_t - q_d)

        q_s = q_d + q_b 

        q_u = 0.0
        return p_star, q_d, q_s, q_b, q_u

    # 2) Check if solar alone is binding but consumers' marginal value at s_t is below utility cost:
    #    v(s_t) < c_u ⇒ no one pays c_u, so price still 0, consumers get only solar, battery can't charge
    v_at_solar = v_max * (1 - s_t / q_max)
    if v_at_solar < c_u:
        p_star = 0.0
        q_d = s_t
        q_s = s_t
        q_b = 0.0        # no excess to charge into battery
        q_u = 0.0
        return p_star, q_d, q_s, q_b, q_u

    # 3) Otherwise utility steps in at p=c_u
    #    Find consumers' demand at that price
    q_d_cu = q_max * (1 - c_u / v_max)
    total_capacity = s_t + q_u_max

    # 3a) If demand > total capacity, clear at choke price > c_u
    if q_d_cu > total_capacity:
        q_d = total_capacity
        p_star = c_u #last marginal unit is solar 
    else:
        # clear at utility price
        q_d = q_d_cu
        p_star = c_u

    # 4) Allocate dispatch:
    #    solar first (always price=0), then utility (battery can't charge at p>0)
    remaining = q_d
    q_s = min(s_t, remaining)
    remaining -= q_s

    q_b = 0.0     # battery bid only fills at p=0
    q_u = min(q_u_max, max(0.0, remaining))  # enforce non-negativity

    return p_star, q_d, q_s, q_b, q_u


# # Example
# if __name__ == "__main__":
#     s_t        = 50.0   # solar
#     q_b_bid    = 20.0   # battery bid at p=0
#     v_max, q_max = 10.0, 100.0
#     c_u, q_u_max = 5.0,  40.0

#     p_star, q_d, q_s, q_b, q_u = equilibrium_with_battery_bid(
#         s_t, q_b_bid, v_max, q_max, c_u, q_u_max
#     )
#     print(f"p*={p_star:.2f}, consumers={q_d:.2f}, solar={q_s:.2f}, battery_buy={q_b:.2f}, utility={q_u:.2f}")


# Example usage:
if __name__ == "__main__":

    s_t        = 5.493672091399086 # solar
    q_b_offer  = 0  # battery offer
    q_b_bid = 10
    v_max, q_max = 10.0, 10.0
    c_u, q_u_max = 5.0,  10

    p_star, q_d, q_s, q_b, q_u = equilibrium_with_battery_offer(
        s_t, q_b_offer, v_max, q_max, c_u, q_u_max
    )

    # p_star, q_d, q_s, q_b, q_u = equilibrium_with_battery_bid(
    #     s_t, q_b_bid, v_max, q_max, c_u, q_u_max
    # )
    print(f"p*={p_star:.2f}, q_d={q_d:.2f}, q_s={q_s:.2f}, q_b={q_b:.2f}, q_u={q_u:.2f}")
    v = v_max*(1- (q_b + q_s)/q_max)
    print(v)


p*=0.00, q_d=5.49, q_s=5.49, q_b=0.00, q_u=0.00
4.506327908600914


## Battery Bellman Dynamic Programming Optimization that works!

In [95]:

days = 7
T = 24 * days 
v_max = 10
q_max = 10
c_u = 5
q_u_max = q_max
C_init = 16
C_max = 20

q_b_max = 2
# Error toleran1e: half your grid spacing
atol = 0
beta = 1 
mean_pmax = 20
std_pmax = 0
sunrise = 6
sunset  = 20

solar_gen_profile, _ = generate_nday_solar(
    sunrise, sunset, mean_pmax, std_pmax=std_pmax, days=days
)
s_t = np.array(solar_gen_profile)
# Setup grids
C_grid = np.arange(0, C_max + 1) #integer 
q_b_grid = np.arange(-q_b_max,q_b_max + 1) #integer 


# Initialize value function arrays
V = np.zeros((len(C_grid), T+1))
P = np.zeros_like(V)
QS = np.zeros_like(V)
QB = np.zeros_like(V)
QU = np.zeros_like(V)

# Terminal constraint: return to initial SOC
V[:, T] = -np.inf
i_init = np.searchsorted(C_grid, C_init)
V[i_init, T] = 0

# Value function iteration loop (same as before)
for t in reversed(range(T)):
    solar = s_t[t]
    for i, C in enumerate(C_grid):
        best = -np.inf
        bp = bqs = bqb = bqu = 0.0
        for q_b_val in q_b_grid:
            C_next = C - q_b_val
            
            if t == T - 1 and not np.isclose(C_next, C_init, atol=0):
                continue
            if not (0 <= C_next <= C_max):
                continue
            
            if q_b_val < 0:
                p_star, q_d, q_s, q_b_ex, q_u = equilibrium_with_battery_bid(
                    solar, -q_b_val, v_max, q_max, c_u, q_u_max
                )
                q_b = -q_b_ex

                # Skip if partial fill, partial fills move the VFI away from the grid 
                if not np.isclose(q_b_ex, -q_b_val, atol=1e-6):
                    continue
                        
            else:
                p_star, q_d, q_s, q_b_ex, q_u = equilibrium_with_battery_offer(
                    solar, q_b_val, v_max, q_max, c_u, q_u_max
                )
                q_b = q_b_ex

                # Skip if partial fill, partial fills move the VFI away from the grid 
                if not np.isclose(q_b_ex, q_b_val, atol=1e-6):
                    continue
            
                
            inst = p_star * q_b
            idx = np.searchsorted(C_grid, C_next)
            idx = min(max(idx, 0), len(C_grid)-1)
            val = inst + beta*V[idx, t+1]
            if val > best:
                best = val
                bp, bqs, bqb, bqu = p_star, q_s, q_b, q_u
        V[i, t] = best
        P[i, t], QS[i, t], QB[i, t], QU[i, t] = bp, bqs, bqb, bqu

# Forward pass for optimal policy
i = i_init
C = C_init
prices = []
socs = []
q_s_list = []
q_b_list = []
q_u_list = []
q_d_list = []
surplus_battery = []
surplus_utility = []
surplus_solar = []
surplus_demand = []
surplus_total = []

for t in range(T):
    socs.append(C)
    prices.append(P[i, t])
    q_s_list.append(QS[i, t])
    q_b_list.append(QB[i, t])
    q_u_list.append(QU[i, t])

    #Determining Demand 
    p = P[i,t] #current price
    q_d = q_max*(1-p/v_max)
    q_d_list.append(q_d)

    # Surplus calculations 
    surplus_battery.append(P[i, t] * QB[i, t])  
    surplus_solar.append(P[i, t] * QS[i, t])   
    surplus_utility.append((P[i, t] - c_u) * QS[i, t])  
    surplus_demand.append(v_max * q_d - (v_max / (2 * q_max)) * q_d**2 - p * q_d)                  
    surplus_total.append(surplus_battery[-1] + surplus_utility[-1] + surplus_demand[-1] + surplus_solar[-1])
    
    # #Advance to next state 
    # C = C - QB[i, t]
    # i = np.searchsorted(C_grid, C)
    # i = min(max(i, 0), len(C_grid)-1)

    #valid 
    q_b_t = QB[i, t]
    C_next = C - q_b_t

    # Clip to [0, C_max] range to ensure SOC validity
    C_next = max(0, min(C_max, C_next))

    # Snap to nearest valid grid point
    i = np.abs(C_grid - C_next).argmin()
    C = C_grid[i]  # Update to snapped value

socs.append(C)

In [96]:
from plotly.subplots import make_subplots


import plotly.graph_objects as go

fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.07,
                    subplot_titles=("Optimal Battery Dispatch", "Battery State-of-Charge (SOC)", "Market Clearing Price"))

# 1. Battery Dispatch
fig.add_trace(go.Scatter(y=q_b_list, mode='lines', name='Optimal Battery Policy'), row=1, col=1)
fig.add_hline(y=0, line_dash="dash", line_color="gray", row=1, col=1)

# 2. Battery SOC
fig.add_trace(go.Scatter(y=socs, mode='lines', name='Battery SOC'), row=2, col=1)
fig.add_hline(y=socs[0], line_dash="dot", line_color="green", row=2, col=1)

# 3. Market Price
fig.add_trace(go.Scatter(y=prices, mode='lines', name='Market Price'), row=3, col=1)

fig.update_yaxes(title_text="Battery Dispatch (q_b)", row=1, col=1)
fig.update_yaxes(title_text="SOC", row=2, col=1)
fig.update_yaxes(title_text="Price", row=3, col=1)
fig.update_xaxes(title_text="Time Step", row=3, col=1)

fig.update_layout(height=800, width=1000, showlegend=True, legend=dict(x=0.85, y=1.05))
fig.show()



In [97]:

import plotly.graph_objects as go

fig = go.Figure()

time_index = list(range(T+1))

fig.add_trace(go.Scatter(x=time_index, y=q_s_list, mode='lines', name='Solar Dispatch', line=dict(color='green')))
fig.add_trace(go.Scatter(x=time_index, y=q_b_list, mode='lines', name='Battery Dispatch', line=dict(color='red')))
fig.add_trace(go.Scatter(x=time_index, y=q_u_list, mode='lines', name='Utility Dispatch', line=dict(color='purple')))
fig.add_trace(go.Scatter(x=time_index, y=q_d_list, mode='lines', name='Total Demand', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=time_index, y=[s for s in s_t], mode='lines', name='Solar Supply', line=dict(color='orange', dash='dash')))

fig.update_layout(
    title="Dispatch and Market Quantities",
    xaxis_title="Time",
    yaxis_title="Quantity"
    
)

fig.show()

## Testing the Simple Code 

In [ ]:

T = 1 
k = 100

reserve_x = 4
reserve_y = k/reserve_x
#Model Initialization

#Demand/Consumer/Load agent 
demand_agent = ElasticDemand(10,10, "demand", True )
utility_agent = Supply(5, 10, "utility", True)
solar_agent = Supply(0, 5, "solar", True )
battery_agent = Battery(2,10,"battery", [5,0],10, 10, 5, True)

amm_agent = AMM()
amm_agent.setup_pool(reserve_x, reserve_y)
agent_list = [demand_agent, utility_agent, solar_agent]
trades_per_period = 100

#Model Agent 
model = Model(T, agent_list, amm_agent, trades_per_period)
model.simulate()


### Market Clearing Price with Battery 

In [ ]:
import numpy as np

def equilibrium_with_battery(v_max, q_max, s_t, C, c_u, q_u_max):
    """
    Compute equilibrium price, quantity, and battery dispatch.

    Demand:      v(q)    = v_max * (1 - q / q_max)
    Solar supply: free up to s_t
    Battery:      free up to C
    Utility:      cost = c_u, capacity = q_u_max

    Returns:
      p_star : float  # equilibrium price
      q_star : float  # equilibrium quantity
      q_b     : float # battery dispatch (sold)
    """
    # 1) Total free supply from solar + battery
    free_supply = s_t + C

    # CASE A: demand at zero price is q_max.
    # If q_max <= s_t, solar alone meets demand -> price=0, battery stays idle
    if q_max <= s_t:
        return 0.0, q_max, 0.0

    # If s_t < q_max <= free_supply, solar+bat cover demand at p=0
    if q_max <= free_supply:
        # battery only dispatches what solar cannot
        q_b = max(0.0, q_max - s_t)
        return 0.0, q_max, q_b

    # CASE B: free supply exhausted, price jumps to c_u
    # quantity demanded at p=c_u
    q_d_at_cu = q_max * (1 - c_u / v_max)

    # total capacity including utility
    total_capacity = free_supply + q_u_max

    if q_d_at_cu <= total_capacity:
        # equilibrium at (c_u, q_d_at_cu)
        # battery dispatch = as much as it can up to C, but not more than q_d_at_cu - s_t
        q_b = min(C, max(0.0, q_d_at_cu - s_t))
        return c_u, q_d_at_cu, q_b

    # CASE C: even at utility cost, demand exceeds total capacity
    # clear at choke price so q* = total_capacity
    q_star = total_capacity
    p_star = v_max * (1 - q_star / q_max)
    # battery dispatches its entire state C
    q_b = C
    return p_star, q_star, q_b



def equilibrium_price_and_quantity(v_max, q_max, s_t, c_u, q_u_max, t):
    """
    Find (p*, q*) solving demand v(q) = supply stack:
      - Solar free up to s_t[t]
      - Utility at cost c_u up to q_u_max
    Demand: v(q)=v_max*(1-q/q_max)
    """

    # 1) If solar alone covers full demand at p=0
    if q_max <= s_t[t]:
        return 0.0, q_max

    # 2) If solar < full demand, but the next marginal value is below c_u:
    #    v(s_t) = v_max*(1 - s_t/q_max) < c_u
    #    ⇒ nobody is willing to pay the utility price, so
    #    market clears on free solar alone
    v_at_solar = v_max * (1 - s_t[t] / q_max)
    if v_at_solar < c_u:
        return 0.0, s_t[t]

    # 3) Otherwise demand at price c_u (utility kicks in)
    q_star = q_max * (1 - c_u / v_max)
    total_supply = s_t[t] + q_u_max

    # 3a) If that demand exceeds total capacity, clear at choke price
    if q_star > total_supply:
        p_star = v_max * (1 - total_supply / q_max)
        return p_star, total_supply

    # 3b) Otherwise clear at (c_u, q_star)
    return c_u, q_star

# Example
if __name__ == "__main__":
    v_max, q_max = 10.0, 100.0  # demand parameters
    s_t     =  30.0            # solar free supply
    C       =  20.0            # battery state-of-charge
    c_u     =   5.0            # utility marginal cost
    q_u_max =  50.0            # utility capacity

    p_star, q_star, q_b = equilibrium_with_battery(
        v_max, q_max, s_t, C, c_u, q_u_max
    )
    print(f"p* = {p_star:.2f}, q* = {q_star:.2f}, battery sells q_b = {q_b:.2f}")


p* = 5.00, q* = 50.00, battery sells q_b = 20.00


### Battery Testing 

In [ ]:
# === Parameters ===
days = 1
T = 24*days + 1
v_max = 10
q_max = 10
c_u = 5
C_init = 2
C_final = 2
C_max = 10

# === Simulated solar irradiance ===
sunrise = 6 
sunset = 20
mean_pmax = 10
std_pmax = 1 


solar_gen_profile, p_max_list = generate_nday_solar(sunrise, sunset, mean_pmax, std_pmax, days)

#np.random.seed(0)
s_t = np.array(solar_gen_profile)  
s_t = np.array([0, 5, 0])

T = 1
k = 10000

reserve_x = 40 #number of Energy Tokens in the pool 
reserve_y = k/reserve_x #number of Money Tokens in the pool
#Model Initialization

#Demand/Consumer/Load agent 
demand_agent = ElasticDemand(10,10, "demand", True )
utility_agent = Supply(5, 10, "utility", True)
solar_agent = Supply(0, 5, "solar", True )
battery_agent = Battery(C_init,C_max,"battery", s_t, v_max, q_max, c_u, verbose= True)

#AMM 
amm_agent = AMM()
amm_agent.setup_pool(reserve_x, reserve_y)
agent_list = [battery_agent]
trades_per_period = 100

#Model Agent 
model = Model(T, agent_list, amm_agent, trades_per_period)
model.simulate()

[demand][INIT] v_max=10, q_max=10, agent_type=demand, verbose=True
[utility][INIT] c=5, q_sell=10, verbose=True
[solar][INIT] c=0, q_sell=5, verbose=True
[battery][init]  C_init = 2 C_max = 10 agent_type = battery
[Model][simulate] Starting simulation for 1 periods, 100 trades per period.

[Model][simulate] === Period 1/1 ===
[Model][simulate] -- Trade 1/100 --
[Model][simulate] Agent: Battery
[Model][simulate] AMM State: E=40, M=250.0
[battery][make_decision] E=40, M=250.0, t=0, p_t=6.25, K=10000.0, p_nxt=5, SOC=2, C_max=10
[battery] p_t=6.2500, p_1=5.0000, SOC=2.0000
[battery][make_decision][SELL] q_uncon=4.721359549995796
[battery][make_decision][SELL] q_req=2.0
[battery][make_decision][SELL] tx={'decision': 'sell_tokens_min_price', 'token': 'x', 'quantity': 2.0, 'min_price': 11.337868480725623, 'agent_type': 'battery'}
[Model][simulate] Agent decision: {'decision': 'sell_tokens_min_price', 'token': 'x', 'quantity': 2.0, 'min_price': 11.337868480725623, 'agent_type': 'battery'}
[Mod

In [ ]:
q_req =  0.472135955

reserve_x = 4 
reserve_y = 25
K = reserve_x*reserve_y
p_nxt = 5 

p_t = reserve_y/reserve_x

p_final = K/(reserve_y- q_req)**2


max_total = p_nxt * q_req #NOTE: think about this 

tx = {
"decision":   "buy_tokens_max_price",
"token":      "x",
"quantity":   q_req,
"max_price":  max_total, 
"agent_type": self.agent_type
}


print(tx)

In [ ]:
T = 1 
#Demand/Consumer/Load agent 
# demand_agent = Demand(10,10, "demand")
# utility_agent = Supply(5, 10, "utility")
# solar_agent = Supply(0, 5, "solar")

# amm_agent = AMM()
# amm_agent.setup_pool(5,5)
# agent_list = [demand_agent,utility_agent, solar_agent]
# trades_per_period = 10


# #Model Agent 
# model = Model(T, agent_list, amm_agent, trades_per_period)

k = 100
param_grid_reserves = []
param_grid_final_reserves = []
T = 1 

for step in range(1,k+1): 
    reserve_x = step
    reserve_y = k/step
    tup = (reserve_x, reserve_y)
    param_grid_reserves.append(tup)

    #Model Initialization
    #Demand/Consumer/Load agent 
    demand_agent = Demand(10,10, "demand")
    utility_agent = Supply(5, 10, "utility")
    solar_agent = Supply(0, 5, "solar")

    amm_agent = AMM()
    amm_agent.setup_pool(reserve_x, reserve_y)
    agent_list = [demand_agent, utility_agent, solar_agent]
    trades_per_period = 100

    #Model Agent 
    model = Model(T, agent_list, amm_agent, trades_per_period)
    model.simulate()
    param_grid_final_reserves.append(model.amm_period_final_state[0])

    
#model.simulate(param_grid_reserves)

In [ ]:
import pandas as pd
df = pd.DataFrame(param_grid_final_reserves, columns=['reserve_x_final', 'reserve_y_final'])
df2 = pd.DataFrame(param_grid_reserves, columns=['reserve_x_initial', 'reserve_y_initial'])

df = df.join(df2)

pd.set_option('display.max_rows', None)

df["x_difference"] = abs(df[f"reserve_x_final"] - df[f"reserve_x_initial"])
df["y_difference"] = abs(df[f"reserve_y_final"] - df[f"reserve_y_initial"])

df["spot_xy_initial"] = df[f"reserve_x_initial"]/df["reserve_y_initial"]
df["spot_xy_final"] = df[f"reserve_x_final"]/df["reserve_y_final"]


df["total_difference"] = df["x_difference"] + df["y_difference"]

min_row = df.loc[df["total_difference"].idxmin()]
print(min_row)

In [ ]:
np.sqrt(100/6.625)

In [ ]:
new_steps = np.linspace(3,5, 100)

param_grid_reserves = []
param_grid_final_reserves = []
T = 1 
k = 100

for step in new_steps: 
    reserve_x = step
    reserve_y = k/step
    tup = (reserve_x, reserve_y)
    param_grid_reserves.append(tup)

    #Model Initialization
    #Demand/Consumer/Load agent 
    demand_agent = Demand(10,10, "demand")
    utility_agent = Supply(5, 10, "utility")
    solar_agent = Supply(0, 5, "solar")
    
    amm_agent = AMM()
    amm_agent.setup_pool(reserve_x, reserve_y)
    agent_list = [demand_agent, utility_agent, solar_agent]
    trades_per_period = 10

    #Model Agent 
    model = Model(T, agent_list, amm_agent, trades_per_period)
    model.simulate()
    param_grid_final_reserves.append(model.amm_period_final_state[0])


In [ ]:
df = pd.DataFrame(param_grid_final_reserves, columns=['reserve_x_final', 'reserve_y_final'])
df2 = pd.DataFrame(param_grid_reserves, columns=['reserve_x_initial', 'reserve_y_initial'])

df = df2.join(df)

pd.set_option('display.max_rows', None)

df["x_difference"] = abs(df[f"reserve_x_final"] - df[f"reserve_x_initial"])
df["y_difference"] = abs(df[f"reserve_y_final"] - df[f"reserve_y_initial"])
df["spot_yx_initial"] = df[f"reserve_y_initial"]/df["reserve_x_initial"]
df["spot_yx_final"] = df[f"reserve_y_final"]/df["reserve_x_final"]

df["total_difference"] = df["x_difference"] + df["y_difference"]

min_row = df.loc[df["total_difference"].idxmin()]
print(min_row)

In [ ]:
df

In [ ]:
# Initialize AMM with some reserves
amm = AMM()
amm.setup_pool(quantity_x=100, quantity_y=100)  # Example: 100 energy tokens, 100 money tokens

# Sell 10 units of energy token 'x' to the AMM
sell_result = amm.sell_tokens(token='x', quantity=10)
print("Sell 10 units of 'x':", sell_result)

# Buy 10 units of energy token 'x' from the AMM
buy_result = amm.buy_tokens(token='x', quantity=10)
print("Buy 10 units of 'x':", buy_result)

# Show final reserves
print("Final AMM reserves:", amm.request_info())

In [ ]:
k = 100 
T = 1
step = 4.47
reserve_x = step 
reserve_y = k/step
tup = (reserve_x, reserve_y)

print(tup)
param_grid_reserves.append(tup)

#Model Initialization
#Demand/Consumer/Load agent 
demand_agent = Demand(10,10, "demand")
utility_agent = Supply(9, 15, "utility")
solar_agent = Supply(0, 5, "solar")

amm_agent = AMM()
amm_agent.setup_pool(reserve_x, reserve_y)
agent_list = [demand_agent, utility_agent, solar_agent]
#agent_list = [demand_agent, utility_agent]
trades_per_period = 15

#Model Agent 
model = Model(T, agent_list, amm_agent, trades_per_period)
model.simulate()
param_grid_final_reserves.append(model.amm_period_final_state[0])



In [ ]:
# Initialize AMM with some reserves
amm = AMM()
amm.setup_pool(quantity_x=250, quantity_y=40)  # Example: 100 energy tokens, 100 money tokens

c = 5
q = 1
# Sell 10 units of energy token 'x' to the AMM
sell_result = amm.sell_tokens_min_price(token='x', quantity = q, min_price= c*q)
print("Sell 10 units of 'x':", sell_result)

# # Buy 10 units of energy token 'x' from the AMM
# buy_result = amm.buy_tokens(token='x', quantity=10)
# print("Buy 10 units of 'x':", buy_result)

# Show final reserves
print("Final AMM reserves:", amm.request_info())

In [ ]:
utility_agent.remaining_supply

In [66]:
import numpy as np

def find_optimal_e(E, M, v_max, q_max):
    """
    Solves for e in:
      e^3 -(q_max + 2E)e^2 + (E^2 + 2E q_max)e + q_max*E*(M/v_max - E) = 0
    Returns the unique real root 0 < e < E.
    """
    # Coefficients of the monic cubic
    coeffs = [
        1,
        -(q_max + 2*E),
        E**2 + 2*E*q_max,
        q_max*E*(M/v_max - E)
    ]
    
    roots = np.roots(coeffs)
    # Filter for a real root in (0, E)
    real_roots = [r.real for r in roots if abs(r.imag) < 1e-8 and 0 < r.real < E]
    if not real_roots:
        raise ValueError("No valid root found in (0, E)")
    return real_roots 



def find_e_ac_closed_form(E, M, v_max, q_max):
    """
    Solve the quadratic
      e^2 - (E + q_max)*e + (E*q_max - (M*q_max)/v_max) = 0
    using the quadratic formula, and return the unique root in (0, E).
    """
    # coefficients
    a = 1.0
    b = -(E + q_max)
    c = E*q_max - (M*q_max)/v_max

    # compute discriminant
    disc = b*b - 4*a*c
    if disc < 0:
        raise ValueError("Discriminant negative: no real roots")

    # two roots
    sqrt_disc = np.sqrt(disc)
    e1 = (-b + sqrt_disc) / (2*a)
    e2 = (-b - sqrt_disc) / (2*a)

    # pick the one in (0, E)
    candidates = [e for e in (e1, e2) if 0 < e < E]
    if not candidates:
        raise ValueError("No valid root in (0, E)")
    return candidates[0]

def compute_consumer_surplus(e, p, v_max, q_max):
    """
    Compute CS = \int_0^e v(q)dq - p*e
      = v_max*(e - e^2/(2*q_max)) - p*e
    """
    area_under = v_max * (e - e**2/(2 * q_max))
    return area_under - p * e

4.4721359549995805,22.360679774997898

# Example
if __name__ == "__main__":
    E, M, v_max, q_max =4.4721359549995805,22.360679774997898, 10.0, 10.0
    e_ac = find_e_ac_closed_form(E, M, v_max, q_max)
   
    # price either via average cost or v(e)
    p_ac = (E*M/(E - e_ac) - M) / e_ac  
    print(f"Average‐cost solution: e = {e_ac:.6f}, price ≃ {p_ac:.6f}")
    cs_avg = compute_consumer_surplus(e_ac,p_ac, v_max, q_max)
    print(f"Average Cost Consumer Surplus = {cs_avg}")
    #marginal price

    e_mc = find_optimal_e(E, M, v_max, q_max)[0]
    p_mc = (E*M/(E - e_mc) - M) / e_mc  
    print(f"Marginal Cost solution e = {e_mc:.6} price ≃ {p_mc:.6f}") 

    cs_mc = compute_consumer_surplus(e_mc,p_mc, v_max, q_max)
    print(f"Marginal Cost Consumer Surplus = {cs_mc}")


Average‐cost solution: e = 0.054324, price ≃ 5.061483
Average Cost Consumer Surplus = 0.13413924915116338
Marginal Cost solution e = 0.0536561 price ≃ 5.060718
Marginal Cost Consumer Surplus = 0.1341600241832297


In [ ]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact, FloatSlider

def demand_curve(q, v_max, q_max):
    """
    Inverse demand (marginal value) function v(q).
    """
    return v_max * (1 - q / q_max)


def amm_supply_curve(q, E, M):
    """
    AMM marginal cost supply curve: price to purchase an incremental unit of E.
    Derived as dM/de = K/(E - e)^2, where K = E*M.
    """
    K = E * M
    return K / (E - q)**2


def plot_demand_and_amm_supply(v_max=0.4, q_max=10.0, price=0.2, E=10.0, M=10.0):
    # Quantity grid up to q_max
    q_vals = np.linspace(0, q_max, 300)

    # Demand curve (marginal value)
    v_vals = demand_curve(q_vals, v_max, q_max)

    # AMM supply (marginal cost)
    supply_vals = amm_supply_curve(q_vals, E, M)

    # Compute consumer surplus at given price
    area, q_star = consumer_surplus(price, v_max, q_max)

    fig = go.Figure()

    # Plot demand curve
    fig.add_trace(go.Scatter(
        x=q_vals, y=v_vals,
        mode='lines', name='Inverse Demand Curve'
    ))

    # Plot AMM supply curve
    fig.add_trace(go.Scatter(
        x=q_vals, y=supply_vals,
        mode='lines', name='AMM Supply Curve',
        line=dict(dash='dot', color='green')
    ))

    # Plot market price line
    fig.add_trace(go.Scatter(
        x=[0, q_max], y=[price, price],
        mode='lines', name='Market Price',
        line=dict(dash='dash', color='red')
    ))

    # Shade consumer surplus
    q_fill = q_vals[q_vals <= q_star]
    v_fill = demand_curve(q_fill, v_max, q_max)
    fig.add_trace(go.Scatter(
        x=np.concatenate([q_fill, q_fill[::-1]]),
        y=np.concatenate([v_fill, [price]*len(q_fill)]),
        fill='toself', fillcolor='rgba(255,165,0,0.5)',
        line=dict(color='orange'), name='Consumer Surplus'
    ))

    fig.update_layout(
        title=f'Consumer Surplus = {area:.4f}, Equilibrium Quantity = {q_star:.2f}',
        xaxis_title='Quantity (q)',
        yaxis_title='Price / Marginal Value',
        xaxis=dict(range=[0, q_max*1.1]),
        yaxis=dict(range=[0, max(v_max, max(supply_vals))*1.1]),
        legend=dict(x=0.01, y=0.99)
    )

    fig.show()

# Interactive sliders to adjust parameters
interact(
    plot_demand_and_amm_supply,
    v_max=FloatSlider(value=0.4, min=0.1, max=10, step=0.01, description='v_max'),
    q_max=FloatSlider(value=10, min=1, max=20, step=0.5, description='q_max'),
    price=FloatSlider(value=0.2, min=0.0, max=10, step=0.01, description='Price'),
    E=FloatSlider(value=10, min=1, max=100, step=1, description='E reserve'),
    M=FloatSlider(value=10, min=1, max=100, step=1, description='M reserve')
)